## CODIGO FINAL

In [1]:
import pandas as pd
import pandas_gbq
from pandas.io import gbq
from datetime import date, timedelta, datetime
import ctypes
import chardet
import gcsfs
from io import StringIO
import google.auth
from gcloud import storage
from oauth2client.service_account import ServiceAccountCredentials
from google.cloud import bigquery
from google.oauth2 import service_account
import logging
import os
import win32com.client
import warnings
warnings.filterwarnings('ignore')
import time

# Credentials to extract data from the Bucket
fs = gcsfs.GCSFileSystem(project='nh-cro-forecast', token = r'C:\Users\ej.garcia\OneDrive - Minor Hotels Europe & Americas\Escritorio\nh-cro-forecast-ec5c044f54cf.json')



#DEFINIMOS AÑO, MES E INICIO DE DIA Y FINAL DE DIA
Year = "2024"
print('Año: ',Year)
Month = "05"
print("Mes: ",Year)

inicio_dia = "1"
print("Día inicio:", inicio_dia)

fin_dia = "23"
print("Día fin:", fin_dia)

# INDICAMOS NUMERO DE QUERY QUE QUEREMOS EJECUTAR

Query_number =  3



if Query_number == 1:

    for i in range(int(inicio_dia),int(fin_dia)+1):
            if i <10: 
                Day = "0"+str(i)
                print(f"Subiendo datos del archivo de día:{Day}")
                print('Traffic_Calls UPLOADING')
                with fs.open('trueit_external/Calls_Distribution/Calls_Distribution_'+Year+Month+Day+'.csv') as f:
                    doc = pd.read_csv(f,delimiter = ';', encoding='latin-1')
                doc.columns=['Date',
                            'Service',
                            'Campaign',
                            'Campaign_number',
                            'Language',
                            'IVR_option',
                            'Time_slot',
                            'Incoming_calls',
                            'Answered_calls',
                            'Abandoned_less_10s',
                            'Abandoned_greater_10s']
                doc["Date"] = pd.to_datetime(doc["Date"], format = '%d/%m/%Y').dt.strftime('%Y-%m-%d')
                Output = doc.astype(str)
                convert_dict = {'Date':object,
                                'Service':object,
                                'Campaign':object,
                                'Campaign_number':object,
                                'Language':object,
                                'IVR_option':object,
                                'Time_slot':object,
                                'Incoming_calls':int,
                                'Answered_calls':int,
                                'Abandoned_less_10s':int,
                                'Abandoned_greater_10s':int,
                                }
                Call_db = Output.astype(convert_dict)
                Call_db = Call_db.astype(str)
                Call_db['Service'] = Call_db['Service'].astype(str)
                Call_db['Incoming_calls'] = Call_db['Incoming_calls'].astype(int)
                Call_db['Answered_calls'] = Call_db['Answered_calls'].astype(int)
                Call_db['Abandoned_less_10s'] = Call_db['Abandoned_less_10s'].astype(int)
                Call_db['Abandoned_greater_10s'] = Call_db['Abandoned_greater_10s'].astype(int)
                Call_db.replace(['NaN','None','NaT',' ','nan'],[None,None,None,None,None],inplace=True)
                # Call_db.to_gbq(destination_table='Evolution.Traffic_Calls',project_id='nh-cro-forecast', if_exists='append')
                
            else:
                Day = str(i)
                print(f"Subiendo datos del archivo de día:{Day}")
                print('Traffic_Calls UPLOADING')
                with fs.open('trueit_external/Calls_Distribution/Calls_Distribution_'+Year+Month+Day+'.csv') as f:
                    doc = pd.read_csv(f,delimiter = ';', encoding='latin-1')
                doc.columns=['Date',
                            'Service',
                            'Campaign',
                            'Campaign_number',
                            'Language',
                            'IVR_option',
                            'Time_slot',
                            'Incoming_calls',
                            'Answered_calls',
                            'Abandoned_less_10s',
                            'Abandoned_greater_10s']
                doc["Date"] = pd.to_datetime(doc["Date"], format = '%d/%m/%Y').dt.strftime('%Y-%m-%d')
                Output = doc.astype(str)
                convert_dict = {'Date':object,
                                'Service':object,
                                'Campaign':object,
                                'Campaign_number':object,
                                'Language':object,
                                'IVR_option':object,
                                'Time_slot':object,
                                'Incoming_calls':int,
                                'Answered_calls':int,
                                'Abandoned_less_10s':int,
                                'Abandoned_greater_10s':int,
                                }
                Call_db = Output.astype(convert_dict)
                Call_db = Call_db.astype(str)
                Call_db['Service'] = Call_db['Service'].astype(str)
                Call_db['Incoming_calls'] = Call_db['Incoming_calls'].astype(int)
                Call_db['Answered_calls'] = Call_db['Answered_calls'].astype(int)
                Call_db['Abandoned_less_10s'] = Call_db['Abandoned_less_10s'].astype(int)
                Call_db['Abandoned_greater_10s'] = Call_db['Abandoned_greater_10s'].astype(int)
                Call_db.replace(['NaN','None','NaT',' ','nan'],[None,None,None,None,None],inplace=True)
                # Call_db.to_gbq(destination_table='Evolution.Traffic_Calls',project_id='nh-cro-forecast', if_exists='append')

elif Query_number == 2:
         for i in range(int(inicio_dia),int(fin_dia)+1):
            if i <10: 
                Day = "0"+str(i)
                print(f"Subiendo datos del archivo de día:{Day}")
                print('Agent_total_breaks UPLOADING')
                with fs.open('trueit_external/Agent_By_Calls_And_Status/Agent_Total_Breaks_'+Year+Month+Day+'.csv') as f:
                    doc = pd.read_csv(f,delimiter = ';', encoding='latin-1')
                doc.columns=['Date',
                            'Agent',
                            'Agent_ID',
                            'Agent_SAP',
                            'Transactions',
                            'Total_calls',
                            'Calls_in',
                            'Calls_out',
                            'Transfer_in',
                            'Transfer_out',
                            'Session',
                            'n_breaks',
                            'Time_break_general',
                            'Available',
                            'Preview',
                            'Time_on_call',
                            'Wrap_up_time',
                            'n_press',
                            'Time_press',
                            'n_sist',
                            'Time_sist',
                            'n_psup',
                            'Time_psup',
                            'n_ndef',
                            'Time_ndef',
                            'n_pfge',
                            'Time_pfge',
                            'n_pterr',
                            'Time_pterr',
                            'Time_administrative',
                            'n_best_buddy_training',
                            'time_best_buddy_training',
                            'n_break',
                            'Time_break',
                            'n_classroom_training',
                            'Time_classroom_training',
                            'n_external_meeting',
                            'Time_external_meeting',
                            'n_internal_meeting',
                            'Time_internal_meeting',
                            'n_manual_dispatching',
                            'Time_manual_dispatching',
                            'n_other_department_gem',
                            'Time_other_department_gem',
                            'n_special_adminwork',
                            'Time_special_adminwork',
                            'n_support',
                            'Time_support',
                            'n_training_on_the_job',
                            'Time_training_on_the_job',
                            'n_visual_rest',
                            'Time_visual_rest',
                            'n_prelogout',
                            'Time_prelogout',
                            'n_unnasociated_dispatching',
                            'Time_unnasociated_dispatching',
                            'n_meeting',
                            'Time_meeting',
                            'n_processing_time',
                            'Time_processing_time',
                            'n_support_supervisor',
                            'Time_support_supervisor',
                            'n_mail_fax',
                            'Time_mail_fax',
                            'n_other_department_individual',
                            'Time_other_deparment_individual',
                            'n_inbox',
                            'Time_inbox',
                            'n_chat',
                            'Time_chat',
                            'n_sdc',
                            'Time_sdc',
                            'n_Followup',
                            'Time_Followup',
                            'Service',
                            'ON_call_hold',
                            'ON_call_active',
                            'N_Assignation',
                            'T_Assignation',
                            'N_Stand_Prio',
                            'T_Stand_Prio',
                            'N_Bulk_and_Prio',
                            'T_Bulk_and_Prio',
                            'N_Overtime',
                            'T_Overtime',
                            'N_Site_Inspection',
                            'T_Site_Inspection', 
                            'N_GEM_Disp', 
                            'T_GEM_Disp',
                            'N_Multidestination',
                            'T_Multidestination',
                            'Number_Audits',
                            'Time_Audits',
                            'Number_Feedbacks',
                            'Time_Feedbacks',
                            'Number_Platforms',
                            'Time_Platforms',
                            'Number_Test',
                            'Time_Test',
                            'Number_Calibracion',
                            'Time_Calibracion', 
                            'Number_Duplicated_y_Spam',
                            'Time_Duplicated_y_Spam',
                            'Number_Break_after_transaction',
                            'Time_Break_after_transaction',
                            'Number_Face_to_Face',
                            'Time_Face_to_Face',
                            'Number_Rest',
                            'Time_Rest']
                doc["Date"] = pd.to_datetime(doc["Date"], format = '%d/%m/%Y').dt.strftime('%Y-%m-%d')
                doc["Session"] = pd.to_datetime(doc["Session"]).dt.strftime('%H:%M:%S')
                doc["Time_break_general"] = pd.to_datetime(doc["Time_break_general"]).dt.strftime('%H:%M:%S')
                doc["Available"] = pd.to_datetime(doc["Available"]).dt.strftime('%H:%M:%S')
                doc["Preview"] = pd.to_datetime(doc["Preview"]).dt.strftime('%H:%M:%S')
                doc["Wrap_up_time"] = pd.to_datetime(doc["Wrap_up_time"]).dt.strftime('%H:%M:%S')
                doc["Time_press"] = pd.to_datetime(doc["Time_press"]).dt.strftime('%H:%M:%S')
                doc["Time_sist"] = pd.to_datetime(doc["Time_sist"]).dt.strftime('%H:%M:%S')
                doc["Time_psup"] = pd.to_datetime(doc["Time_psup"]).dt.strftime('%H:%M:%S')
                doc["Time_ndef"] = pd.to_datetime(doc["Time_ndef"]).dt.strftime('%H:%M:%S')
                doc["Time_pfge"] = pd.to_datetime(doc["Time_pfge"]).dt.strftime('%H:%M:%S')
                doc["Time_pterr"] = pd.to_datetime(doc["Time_pterr"]).dt.strftime('%H:%M:%S')
                doc["Time_pterr"] = pd.to_datetime(doc["Time_pterr"]).dt.strftime('%H:%M:%S')
                doc["Time_administrative"] = pd.to_datetime(doc["Time_administrative"]).dt.strftime('%H:%M:%S')
                doc["time_best_buddy_training"] = pd.to_datetime(doc["time_best_buddy_training"]).dt.strftime('%H:%M:%S')
                doc["Time_break"] = pd.to_datetime(doc["Time_break"]).dt.strftime('%H:%M:%S')
                doc["Time_classroom_training"] = pd.to_datetime(doc["Time_classroom_training"]).dt.strftime('%H:%M:%S')
                doc["Time_external_meeting"] = pd.to_datetime(doc["Time_external_meeting"]).dt.strftime('%H:%M:%S')
                doc["Time_internal_meeting"] = pd.to_datetime(doc["Time_internal_meeting"]).dt.strftime('%H:%M:%S')
                doc["Time_manual_dispatching"] = pd.to_datetime(doc["Time_manual_dispatching"]).dt.strftime('%H:%M:%S')
                doc["Time_other_department_gemsion"] = pd.to_datetime(doc["Time_other_department_gem"]).dt.strftime('%H:%M:%S')
                doc["Time_special_adminwork"] = pd.to_datetime(doc["Time_special_adminwork"]).dt.strftime('%H:%M:%S')
                doc["Time_support"] = pd.to_datetime(doc["Time_support"]).dt.strftime('%H:%M:%S')
                doc["Time_training_on_the_job"] = pd.to_datetime(doc["Time_training_on_the_job"]).dt.strftime('%H:%M:%S')
                doc["Time_visual_rest"] = pd.to_datetime(doc["Time_visual_rest"]).dt.strftime('%H:%M:%S')
                doc["Time_prelogout"] = pd.to_datetime(doc["Time_prelogout"]).dt.strftime('%H:%M:%S')
                doc["Time_unnasociated_dispatching"] = pd.to_datetime(doc["Time_unnasociated_dispatching"]).dt.strftime('%H:%M:%S')
                doc["Time_meeting"] = pd.to_datetime(doc["Time_meeting"]).dt.strftime('%H:%M:%S')
                doc["Time_processing_time"] = pd.to_datetime(doc["Time_processing_time"]).dt.strftime('%H:%M:%S')
                doc["Time_support_supervisor"] = pd.to_datetime(doc["Time_support_supervisor"]).dt.strftime('%H:%M:%S')
                doc["Time_mail_fax"] = pd.to_datetime(doc["Time_mail_fax"]).dt.strftime('%H:%M:%S')
                doc["Time_other_deparment_individual"] = pd.to_datetime(doc["Time_other_deparment_individual"]).dt.strftime('%H:%M:%S')
                doc["Time_inbox"] = pd.to_datetime(doc["Time_inbox"]).dt.strftime('%H:%M:%S')
                doc["Time_chat"] = pd.to_datetime(doc["Time_chat"]).dt.strftime('%H:%M:%S')
                doc["Time_sdc"] = pd.to_datetime(doc["Time_sdc"]).dt.strftime('%H:%M:%S')
                doc["Time_Followup"] = pd.to_datetime(doc["Time_Followup"]).dt.strftime('%H:%M:%S')
                doc["ON_call_hold"] = pd.to_datetime(doc["ON_call_hold"]).dt.strftime('%H:%M:%S')
                doc["ON_call_active"] = pd.to_datetime(doc["ON_call_active"]).dt.strftime('%H:%M:%S')
                doc["Time_on_call"] = pd.to_datetime(doc["Time_on_call"]).dt.strftime('%H:%M:%S')
                doc["T_Assignation"] = pd.to_datetime(doc["T_Assignation"]).dt.strftime('%H:%M:%S')
                doc["T_Stand_Prio"] = pd.to_datetime(doc["T_Stand_Prio"]).dt.strftime('%H:%M:%S')
                doc["T_Bulk_and_Prio"] = pd.to_datetime(doc["T_Bulk_and_Prio"]).dt.strftime('%H:%M:%S')
                doc["T_Overtime"] = pd.to_datetime(doc["T_Overtime"]).dt.strftime('%H:%M:%S')
                doc['T_Site_Inspection'] = pd.to_datetime(doc["T_Site_Inspection"]).dt.strftime('%H:%M:%S')
                doc['T_GEM_Disp'] = pd.to_datetime(doc['T_GEM_Disp']).dt.strftime('%H:%M:%S')
                doc['T_Multidestination'] = pd.to_datetime(doc['T_Multidestination']).dt.strftime('%H:%M:%S')
                doc['Time_Audits'] = pd.to_datetime(doc['Time_Audits']).dt.strftime('%H:%M:%S')
                doc['Time_Feedbacks'] = pd.to_datetime(doc['Time_Feedbacks']).dt.strftime('%H:%M:%S')
                doc['Time_Platforms'] = pd.to_datetime(doc['Time_Platforms']).dt.strftime('%H:%M:%S')
                doc['Time_Test'] = pd.to_datetime(doc['Time_Test']).dt.strftime('%H:%M:%S')
                doc['Time_Calibracion'] = pd.to_datetime(doc['Time_Calibracion']).dt.strftime('%H:%M:%S')
                doc['Time_Duplicated_y_Spam'] = pd.to_datetime(doc['Time_Duplicated_y_Spam']).dt.strftime('%H:%M:%S')
                doc['Time_Break_after_transaction'] = pd.to_datetime(doc['Time_Break_after_transaction']).dt.strftime('%H:%M:%S')
                doc['Time_Face_to_Face'] = pd.to_datetime(doc['Time_Face_to_Face']).dt.strftime('%H:%M:%S')
                doc['Time_Rest'] = pd.to_datetime(doc['Time_Rest']).dt.strftime('%H:%M:%S')
                Output = doc.astype(str)
                convert_dict = {'Date':object,
                                'Agent':object,
                                'Agent_ID':object,
                                'Agent_SAP':object,
                                'Transactions':object,
                                'Total_calls':object,
                                'Calls_in':object,
                                'Calls_out':object,
                                'Transfer_in':object,
                                'Transfer_out':object,
                                'Session':object,
                                'n_breaks':object,
                                'Time_break_general':object,
                                'Available':object,
                                'Preview':object,
                                'Time_on_call':object,
                                'Wrap_up_time':object,
                                'n_press':object,
                                'Time_press':object,
                                'n_sist':object,
                                'Time_sist':object,
                                'n_psup':object,
                                'Time_psup':object,
                                'n_ndef':object,
                                'Time_ndef':object,
                                'n_pfge':object,
                                'Time_pfge':object,
                                'n_pterr':object,
                                'Time_pterr':object,
                                'Time_administrative':object,
                                'n_best_buddy_training':object,
                                'time_best_buddy_training':object,
                                'n_break':object,
                                'Time_break':object,
                                'n_classroom_training':object,
                                'Time_classroom_training':object,
                                'n_external_meeting':object,
                                'Time_external_meeting':object,
                                'n_internal_meeting':object,
                                'Time_internal_meeting':object,
                                'n_manual_dispatching':object,
                                'Time_manual_dispatching':object,
                                'n_other_department_gem':object,
                                'Time_other_department_gem':object,
                                'n_special_adminwork':object,
                                'Time_special_adminwork':object,
                                'n_support':object,
                                'Time_support':object,
                                'n_training_on_the_job':object,
                                'Time_training_on_the_job':object,
                                'n_visual_rest':object,
                                'Time_visual_rest':object,
                                'n_prelogout':object,
                                'Time_prelogout':object,
                                'n_unnasociated_dispatching':object,
                                'Time_unnasociated_dispatching':object,
                                'n_meeting':object,
                                'Time_meeting':object,
                                'n_processing_time':object,
                                'Time_processing_time':object,
                                'n_support_supervisor':object,
                                'Time_support_supervisor':object,
                                'n_mail_fax':object,
                                'Time_mail_fax':object,
                                'n_other_department_individual':object,
                                'Time_other_deparment_individual':object,
                                'n_inbox':object,
                                'Time_inbox':object,
                                'n_chat':object,
                                'Time_chat':object,
                                'n_sdc':object,
                                'Time_sdc':object,
                                'n_Followup':object,
                                'Time_Followup':object,
                                'Service':object,
                                'ON_call_hold':object,
                                'ON_call_active':object,
                                'N_Assignation': object,
                                'T_Assignation': object,
                                'N_Stand_Prio': object,
                                'T_Stand_Prio':object,
                                'N_Bulk_and_Prio':object,
                                'T_Bulk_and_Prio':object,
                                'N_Overtime':object,
                                'T_Overtime':object,
                                'N_Site_Inspection':object,
                                'T_Site_Inspection':object, 
                                'N_GEM_Disp':object, 
                                'T_GEM_Disp':object,
                                'N_Multidestination':object,
                                'T_Multidestination':object,
                                'Number_Audits':object,
                                'Time_Audits':object,
                                'Number_Feedbacks':object,
                                'Time_Feedbacks':object,
                                'Number_Platforms':object,
                                'Time_Platforms':object,
                                'Number_Test':object,
                                'Time_Test':object,
                                'Number_Calibracion':object,
                                'Time_Calibracion':object, 
                                'Number_Duplicated_y_Spam':object,
                                'Time_Duplicated_y_Spam':object,
                                'Number_Break_after_transaction':object,
                                'Time_Break_after_transaction':object,
                                'Number_Face_to_Face':object,
                                'Time_Face_to_Face':object,
                                'Number_Rest':object,
                                'Time_Rest':object} 
                Output["Time_other_department_gem"] = pd.to_datetime(Output["Time_other_department_gem"]).dt.strftime('%H:%M:%S')
                d = []
                for i in range(0, len(Output["Agent"])):
                    d.append(Output["Agent"][i][:-1])
                Output["Agent"] = d
                Call_db = Output.astype(convert_dict)
                Call_db = Call_db.iloc[:,:-1]
                Call_db = Call_db.astype(str)
                Call_db.replace(['NaN','None','NaT',' ','nan'],[None,None,None,None,None],inplace = True)
                Call_db.to_gbq(destination_table='Evolution.Agent_total_breaks',project_id='nh-cro-forecast', if_exists='append')

            else:
                Day = str(i)
                print(f"Subiendo datos del archivo de día:{Day}")
                print('Agent_total_breaks UPLOADING')
                with fs.open('trueit_external/Agent_By_Calls_And_Status/Agent_Total_Breaks_'+Year+Month+Day+'.csv') as f:
                    doc = pd.read_csv(f,delimiter = ';', encoding='latin-1')
                doc.columns=['Date',
                            'Agent',
                            'Agent_ID',
                            'Agent_SAP',
                            'Transactions',
                            'Total_calls',
                            'Calls_in',
                            'Calls_out',
                            'Transfer_in',
                            'Transfer_out',
                            'Session',
                            'n_breaks',
                            'Time_break_general',
                            'Available',
                            'Preview',
                            'Time_on_call',
                            'Wrap_up_time',
                            'n_press',
                            'Time_press',
                            'n_sist',
                            'Time_sist',
                            'n_psup',
                            'Time_psup',
                            'n_ndef',
                            'Time_ndef',
                            'n_pfge',
                            'Time_pfge',
                            'n_pterr',
                            'Time_pterr',
                            'Time_administrative',
                            'n_best_buddy_training',
                            'time_best_buddy_training',
                            'n_break',
                            'Time_break',
                            'n_classroom_training',
                            'Time_classroom_training',
                            'n_external_meeting',
                            'Time_external_meeting',
                            'n_internal_meeting',
                            'Time_internal_meeting',
                            'n_manual_dispatching',
                            'Time_manual_dispatching',
                            'n_other_department_gem',
                            'Time_other_department_gem',
                            'n_special_adminwork',
                            'Time_special_adminwork',
                            'n_support',
                            'Time_support',
                            'n_training_on_the_job',
                            'Time_training_on_the_job',
                            'n_visual_rest',
                            'Time_visual_rest',
                            'n_prelogout',
                            'Time_prelogout',
                            'n_unnasociated_dispatching',
                            'Time_unnasociated_dispatching',
                            'n_meeting',
                            'Time_meeting',
                            'n_processing_time',
                            'Time_processing_time',
                            'n_support_supervisor',
                            'Time_support_supervisor',
                            'n_mail_fax',
                            'Time_mail_fax',
                            'n_other_department_individual',
                            'Time_other_deparment_individual',
                            'n_inbox',
                            'Time_inbox',
                            'n_chat',
                            'Time_chat',
                            'n_sdc',
                            'Time_sdc',
                            'n_Followup',
                            'Time_Followup',
                            'Service',
                            'ON_call_hold',
                            'ON_call_active',
                            'N_Assignation',
                            'T_Assignation',
                            'N_Stand_Prio',
                            'T_Stand_Prio',
                            'N_Bulk_and_Prio',
                            'T_Bulk_and_Prio',
                            'N_Overtime',
                            'T_Overtime',
                            'N_Site_Inspection',
                            'T_Site_Inspection', 
                            'N_GEM_Disp', 
                            'T_GEM_Disp',
                            'N_Multidestination',
                            'T_Multidestination',
                            'Number_Audits',
                            'Time_Audits',
                            'Number_Feedbacks',
                            'Time_Feedbacks',
                            'Number_Platforms',
                            'Time_Platforms',
                            'Number_Test',
                            'Time_Test',
                            'Number_Calibracion',
                            'Time_Calibracion', 
                            'Number_Duplicated_y_Spam',
                            'Time_Duplicated_y_Spam',
                            'Number_Break_after_transaction',
                            'Time_Break_after_transaction',
                            'Number_Face_to_Face',
                            'Time_Face_to_Face',
                            'Number_Rest',
                            'Time_Rest']
                doc["Date"] = pd.to_datetime(doc["Date"], format = '%d/%m/%Y').dt.strftime('%Y-%m-%d')
                doc["Session"] = pd.to_datetime(doc["Session"]).dt.strftime('%H:%M:%S')
                doc["Time_break_general"] = pd.to_datetime(doc["Time_break_general"]).dt.strftime('%H:%M:%S')
                doc["Available"] = pd.to_datetime(doc["Available"]).dt.strftime('%H:%M:%S')
                doc["Preview"] = pd.to_datetime(doc["Preview"]).dt.strftime('%H:%M:%S')
                doc["Wrap_up_time"] = pd.to_datetime(doc["Wrap_up_time"]).dt.strftime('%H:%M:%S')
                doc["Time_press"] = pd.to_datetime(doc["Time_press"]).dt.strftime('%H:%M:%S')
                doc["Time_sist"] = pd.to_datetime(doc["Time_sist"]).dt.strftime('%H:%M:%S')
                doc["Time_psup"] = pd.to_datetime(doc["Time_psup"]).dt.strftime('%H:%M:%S')
                doc["Time_ndef"] = pd.to_datetime(doc["Time_ndef"]).dt.strftime('%H:%M:%S')
                doc["Time_pfge"] = pd.to_datetime(doc["Time_pfge"]).dt.strftime('%H:%M:%S')
                doc["Time_pterr"] = pd.to_datetime(doc["Time_pterr"]).dt.strftime('%H:%M:%S')
                doc["Time_pterr"] = pd.to_datetime(doc["Time_pterr"]).dt.strftime('%H:%M:%S')
                doc["Time_administrative"] = pd.to_datetime(doc["Time_administrative"]).dt.strftime('%H:%M:%S')
                doc["time_best_buddy_training"] = pd.to_datetime(doc["time_best_buddy_training"]).dt.strftime('%H:%M:%S')
                doc["Time_break"] = pd.to_datetime(doc["Time_break"]).dt.strftime('%H:%M:%S')
                doc["Time_classroom_training"] = pd.to_datetime(doc["Time_classroom_training"]).dt.strftime('%H:%M:%S')
                doc["Time_external_meeting"] = pd.to_datetime(doc["Time_external_meeting"]).dt.strftime('%H:%M:%S')
                doc["Time_internal_meeting"] = pd.to_datetime(doc["Time_internal_meeting"]).dt.strftime('%H:%M:%S')
                doc["Time_manual_dispatching"] = pd.to_datetime(doc["Time_manual_dispatching"]).dt.strftime('%H:%M:%S')
                doc["Time_other_department_gemsion"] = pd.to_datetime(doc["Time_other_department_gem"]).dt.strftime('%H:%M:%S')
                doc["Time_special_adminwork"] = pd.to_datetime(doc["Time_special_adminwork"]).dt.strftime('%H:%M:%S')
                doc["Time_support"] = pd.to_datetime(doc["Time_support"]).dt.strftime('%H:%M:%S')
                doc["Time_training_on_the_job"] = pd.to_datetime(doc["Time_training_on_the_job"]).dt.strftime('%H:%M:%S')
                doc["Time_visual_rest"] = pd.to_datetime(doc["Time_visual_rest"]).dt.strftime('%H:%M:%S')
                doc["Time_prelogout"] = pd.to_datetime(doc["Time_prelogout"]).dt.strftime('%H:%M:%S')
                doc["Time_unnasociated_dispatching"] = pd.to_datetime(doc["Time_unnasociated_dispatching"]).dt.strftime('%H:%M:%S')
                doc["Time_meeting"] = pd.to_datetime(doc["Time_meeting"]).dt.strftime('%H:%M:%S')
                doc["Time_processing_time"] = pd.to_datetime(doc["Time_processing_time"]).dt.strftime('%H:%M:%S')
                doc["Time_support_supervisor"] = pd.to_datetime(doc["Time_support_supervisor"]).dt.strftime('%H:%M:%S')
                doc["Time_mail_fax"] = pd.to_datetime(doc["Time_mail_fax"]).dt.strftime('%H:%M:%S')
                doc["Time_other_deparment_individual"] = pd.to_datetime(doc["Time_other_deparment_individual"]).dt.strftime('%H:%M:%S')
                doc["Time_inbox"] = pd.to_datetime(doc["Time_inbox"]).dt.strftime('%H:%M:%S')
                doc["Time_chat"] = pd.to_datetime(doc["Time_chat"]).dt.strftime('%H:%M:%S')
                doc["Time_sdc"] = pd.to_datetime(doc["Time_sdc"]).dt.strftime('%H:%M:%S')
                doc["Time_Followup"] = pd.to_datetime(doc["Time_Followup"]).dt.strftime('%H:%M:%S')
                doc["ON_call_hold"] = pd.to_datetime(doc["ON_call_hold"]).dt.strftime('%H:%M:%S')
                doc["ON_call_active"] = pd.to_datetime(doc["ON_call_active"]).dt.strftime('%H:%M:%S')
                doc["Time_on_call"] = pd.to_datetime(doc["Time_on_call"]).dt.strftime('%H:%M:%S')
                doc["T_Assignation"] = pd.to_datetime(doc["T_Assignation"]).dt.strftime('%H:%M:%S')
                doc["T_Stand_Prio"] = pd.to_datetime(doc["T_Stand_Prio"]).dt.strftime('%H:%M:%S')
                doc["T_Bulk_and_Prio"] = pd.to_datetime(doc["T_Bulk_and_Prio"]).dt.strftime('%H:%M:%S')
                doc["T_Overtime"] = pd.to_datetime(doc["T_Overtime"]).dt.strftime('%H:%M:%S')
                doc['T_Site_Inspection'] = pd.to_datetime(doc["T_Site_Inspection"]).dt.strftime('%H:%M:%S')
                doc['T_GEM_Disp'] = pd.to_datetime(doc['T_GEM_Disp']).dt.strftime('%H:%M:%S')
                doc['T_Multidestination'] = pd.to_datetime(doc['T_Multidestination']).dt.strftime('%H:%M:%S')
                doc['Time_Audits'] = pd.to_datetime(doc['Time_Audits']).dt.strftime('%H:%M:%S')
                doc['Time_Feedbacks'] = pd.to_datetime(doc['Time_Feedbacks']).dt.strftime('%H:%M:%S')
                doc['Time_Platforms'] = pd.to_datetime(doc['Time_Platforms']).dt.strftime('%H:%M:%S')
                doc['Time_Test'] = pd.to_datetime(doc['Time_Test']).dt.strftime('%H:%M:%S')
                doc['Time_Calibracion'] = pd.to_datetime(doc['Time_Calibracion']).dt.strftime('%H:%M:%S')
                doc['Time_Duplicated_y_Spam'] = pd.to_datetime(doc['Time_Duplicated_y_Spam']).dt.strftime('%H:%M:%S')
                doc['Time_Break_after_transaction'] = pd.to_datetime(doc['Time_Break_after_transaction']).dt.strftime('%H:%M:%S')
                doc['Time_Face_to_Face'] = pd.to_datetime(doc['Time_Face_to_Face']).dt.strftime('%H:%M:%S')
                doc['Time_Rest'] = pd.to_datetime(doc['Time_Rest']).dt.strftime('%H:%M:%S')
                Output = doc.astype(str)
                convert_dict = {'Date':object,
                                'Agent':object,
                                'Agent_ID':object,
                                'Agent_SAP':object,
                                'Transactions':object,
                                'Total_calls':object,
                                'Calls_in':object,
                                'Calls_out':object,
                                'Transfer_in':object,
                                'Transfer_out':object,
                                'Session':object,
                                'n_breaks':object,
                                'Time_break_general':object,
                                'Available':object,
                                'Preview':object,
                                'Time_on_call':object,
                                'Wrap_up_time':object,
                                'n_press':object,
                                'Time_press':object,
                                'n_sist':object,
                                'Time_sist':object,
                                'n_psup':object,
                                'Time_psup':object,
                                'n_ndef':object,
                                'Time_ndef':object,
                                'n_pfge':object,
                                'Time_pfge':object,
                                'n_pterr':object,
                                'Time_pterr':object,
                                'Time_administrative':object,
                                'n_best_buddy_training':object,
                                'time_best_buddy_training':object,
                                'n_break':object,
                                'Time_break':object,
                                'n_classroom_training':object,
                                'Time_classroom_training':object,
                                'n_external_meeting':object,
                                'Time_external_meeting':object,
                                'n_internal_meeting':object,
                                'Time_internal_meeting':object,
                                'n_manual_dispatching':object,
                                'Time_manual_dispatching':object,
                                'n_other_department_gem':object,
                                'Time_other_department_gem':object,
                                'n_special_adminwork':object,
                                'Time_special_adminwork':object,
                                'n_support':object,
                                'Time_support':object,
                                'n_training_on_the_job':object,
                                'Time_training_on_the_job':object,
                                'n_visual_rest':object,
                                'Time_visual_rest':object,
                                'n_prelogout':object,
                                'Time_prelogout':object,
                                'n_unnasociated_dispatching':object,
                                'Time_unnasociated_dispatching':object,
                                'n_meeting':object,
                                'Time_meeting':object,
                                'n_processing_time':object,
                                'Time_processing_time':object,
                                'n_support_supervisor':object,
                                'Time_support_supervisor':object,
                                'n_mail_fax':object,
                                'Time_mail_fax':object,
                                'n_other_department_individual':object,
                                'Time_other_deparment_individual':object,
                                'n_inbox':object,
                                'Time_inbox':object,
                                'n_chat':object,
                                'Time_chat':object,
                                'n_sdc':object,
                                'Time_sdc':object,
                                'n_Followup':object,
                                'Time_Followup':object,
                                'Service':object,
                                'ON_call_hold':object,
                                'ON_call_active':object,
                                'N_Assignation': object,
                                'T_Assignation': object,
                                'N_Stand_Prio': object,
                                'T_Stand_Prio':object,
                                'N_Bulk_and_Prio':object,
                                'T_Bulk_and_Prio':object,
                                'N_Overtime':object,
                                'T_Overtime':object,
                                'N_Site_Inspection':object,
                                'T_Site_Inspection':object, 
                                'N_GEM_Disp':object, 
                                'T_GEM_Disp':object,
                                'N_Multidestination':object,
                                'T_Multidestination':object,
                                'Number_Audits':object,
                                'Time_Audits':object,
                                'Number_Feedbacks':object,
                                'Time_Feedbacks':object,
                                'Number_Platforms':object,
                                'Time_Platforms':object,
                                'Number_Test':object,
                                'Time_Test':object,
                                'Number_Calibracion':object,
                                'Time_Calibracion':object, 
                                'Number_Duplicated_y_Spam':object,
                                'Time_Duplicated_y_Spam':object,
                                'Number_Break_after_transaction':object,
                                'Time_Break_after_transaction':object,
                                'Number_Face_to_Face':object,
                                'Time_Face_to_Face':object,
                                'Number_Rest':object,
                                'Time_Rest':object} 
                Output["Time_other_department_gem"] = pd.to_datetime(Output["Time_other_department_gem"]).dt.strftime('%H:%M:%S')
                d = []
                for i in range(0, len(Output["Agent"])):
                    d.append(Output["Agent"][i][:-1])
                Output["Agent"] = d
                Call_db = Output.astype(convert_dict)
                Call_db = Call_db.iloc[:,:-1]
                Call_db = Call_db.astype(str)
                Call_db.replace(['NaN','None','NaT',' ','nan'],[None,None,None,None,None],inplace = True)
                Call_db.to_gbq(destination_table='Evolution.Agent_total_breaks',project_id='nh-cro-forecast', if_exists='append')
        
elif Query_number == 3:
        for i in range(int(inicio_dia),int(fin_dia)+1):
            if i <10: 
                Day = "0"+str(i)
                print(f"Subiendo datos del archivo de día:{Day}")
                print('Calls_per_queue UPLOADING')
                with fs.open('trueit_external/Calls_By_Agent_Campaign/CDR_by_Agent_Campaign_Detail_'+Year+Month+Day+'.csv') as f:
                    doc = pd.read_csv(f,delimiter = ';', encoding='latin-1')
                doc = doc.iloc[0:,1:]
                doc.columns = ['Service',
                            'Date',
                            'Agent',
                            'Agent_ID',
                            'Campaign',
                            'Campaign_number',
                            'Language',
                            'IVR_option',
                            'Calls_in',
                            'Calls_out',
                            'Talk_time_in',
                            'Talk_time_out',
                            'Total_talk_time']
                doc["Date"] = pd.to_datetime(doc["Date"], format = '%d/%m/%Y').dt.strftime('%Y-%m-%d')
                doc["Talk_time_in"] = pd.to_datetime(doc["Talk_time_in"]).dt.strftime('%H:%M:%S')
                doc["Talk_time_out"] = pd.to_datetime(doc["Talk_time_out"]).dt.strftime('%H:%M:%S')
                doc["Total_talk_time"] = pd.to_datetime(doc["Total_talk_time"]).dt.strftime('%H:%M:%S')
                doc['Total_calls'] = doc['Calls_in'].astype(int) + doc['Calls_out'].astype(int)
                Output = doc.astype(str)
                Output
                convert_dict = {'Date':object,
                                'Service':object,
                                'Campaign':object,
                                'Campaign_number':object,
                                'Language':object,
                                'IVR_option':object,
                                'Agent':object,
                                'Agent_ID':object,
                                'Calls_in':object,
                                'Calls_out':object,
                                'Total_calls':object,
                                'Talk_time_in':object,
                                'Talk_time_out':object,
                                'Total_talk_time':object,
                                } 
                Call_db = Output.astype(convert_dict)
                Call_db = Call_db[['Date', 'Service', 'Campaign', 'Campaign_number', 'Language', 'IVR_option','Agent', 'Agent_ID', 'Calls_in', 'Calls_out'
                        , 'Total_calls', 'Talk_time_in', 'Talk_time_out', 'Total_talk_time']]
                Call_db = Call_db.astype(str)
                Call_db.replace(['NaN','None','NaT',' ','nan'],[None,None,None,None,None], inplace=True)
                Call_db.to_gbq(destination_table='Evolution.Calls_per_queue',project_id='nh-cro-forecast', if_exists='append')
            else:
                Day = str(i)
                print(f"Subiendo datos del archivo de día:{Day}")
                print('Calls_per_queue UPLOADING')
                with fs.open('trueit_external/Calls_By_Agent_Campaign/CDR_by_Agent_Campaign_Detail_'+Year+Month+Day+'.csv') as f:
                    doc = pd.read_csv(f,delimiter = ';', encoding='latin-1')
                doc = doc.iloc[0:,1:]
                doc.columns = ['Service',
                            'Date',
                            'Agent',
                            'Agent_ID',
                            'Campaign',
                            'Campaign_number',
                            'Language',
                            'IVR_option',
                            'Calls_in',
                            'Calls_out',
                            'Talk_time_in',
                            'Talk_time_out',
                            'Total_talk_time']
                doc["Date"] = pd.to_datetime(doc["Date"], format = '%d/%m/%Y').dt.strftime('%Y-%m-%d')
                doc["Talk_time_in"] = pd.to_datetime(doc["Talk_time_in"]).dt.strftime('%H:%M:%S')
                doc["Talk_time_out"] = pd.to_datetime(doc["Talk_time_out"]).dt.strftime('%H:%M:%S')
                doc["Total_talk_time"] = pd.to_datetime(doc["Total_talk_time"]).dt.strftime('%H:%M:%S')
                doc['Total_calls'] = doc['Calls_in'].astype(int) + doc['Calls_out'].astype(int)
                Output = doc.astype(str)
                Output
                convert_dict = {'Date':object,
                                'Service':object,
                                'Campaign':object,
                                'Campaign_number':object,
                                'Language':object,
                                'IVR_option':object,
                                'Agent':object,
                                'Agent_ID':object,
                                'Calls_in':object,
                                'Calls_out':object,
                                'Total_calls':object,
                                'Talk_time_in':object,
                                'Talk_time_out':object,
                                'Total_talk_time':object,
                                } 
                Call_db = Output.astype(convert_dict)
                Call_db = Call_db[['Date', 'Service', 'Campaign', 'Campaign_number', 'Language', 'IVR_option','Agent', 'Agent_ID', 'Calls_in', 'Calls_out'
                        , 'Total_calls', 'Talk_time_in', 'Talk_time_out', 'Total_talk_time']]
                Call_db = Call_db.astype(str)
                Call_db.replace(['NaN','None','NaT',' ','nan'],[None,None,None,None,None], inplace=True)
                Call_db.to_gbq(destination_table='Evolution.Calls_per_queue',project_id='nh-cro-forecast', if_exists='append')
elif Query_number == 4:
        for i in range(int(inicio_dia),int(fin_dia)+1):
            if i <10: 
                Day = "0"+str(i)
                print(f"Subiendo datos del archivo de día:{Day}")
                print('SLA_completed UPLOADING')
                with fs.open('trueit_external/SLA_completed/SLA_Completed_'+Year+Month+Day+'.csv') as f:
                    doc = pd.read_csv(f,delimiter = ';', encoding='latin-1')
                File = doc
                File.columns=['group',
                            'Service',
                            'Entry_date',
                            'Entry_hour',
                            'Activity_ID',
                            'Case_ID',
                            'Alias',
                            'Agent_distributed',
                            'Distributed_date',
                            'Distributed_hour',
                            'Queue',
                            'Completed',
                            'Agent_completed',
                            'Completed_date',
                            'Completed_hour',
                            'Type',
                            'Subtype',
                            'due_on',
                            'From_mail',
                            'IN_OUT',
                            'Total_time']
                b = []
                for i in range(len(File["due_on"])):       
                    b.append(File["due_on"][i][File["due_on"][i].rfind(" ")+1:])
                File["Due_on_hour"] = b
                c = []
                for i in range(len(File["due_on"])):
                    c.append(File["due_on"][i][:File["due_on"][i].rfind(" ")])
                File["Due_on_date"] = c
                File = File.iloc[1:len(File)]
                File.drop(['group'],axis=1,inplace=True)
                File["Entry_date"] = pd.to_datetime(File["Entry_date"], format = '%d/%m/%Y').dt.strftime('%Y-%m-%d')
                File["Completed_date"] = pd.to_datetime(File["Completed_date"], format = '%d/%m/%Y').dt.strftime('%Y-%m-%d')
                File["Distributed_date"] = pd.to_datetime(File["Distributed_date"], format = '%d/%m/%Y').dt.strftime('%Y-%m-%d')
                File.drop(columns=['due_on'],inplace = True)
                Output = File.astype(str)
                convert_dict = {'Service':object,
                                'Entry_date':object,
                                'Entry_hour':object,
                                'Activity_ID':object,
                                'Case_ID':object,
                                'Alias':object,
                                'Agent_distributed':object,
                                'Distributed_date':object,
                                'Distributed_hour':object,
                                'Queue':object,
                                'Completed':object,
                                'Agent_completed':object,
                                'Completed_date':object,
                                'Completed_hour':object,
                                'Type':object,
                                'Subtype':object,
                                'From_mail':object,
                                'IN_OUT':object,
                                'Total_time':object,
                                'Due_on_hour':object,
                                'Due_on_date':object} 
                Call_db = Output.astype(convert_dict)
                Call_db = Call_db.reindex(columns = ['Service',
                                                    'Entry_date',
                                                    'Entry_hour',
                                                    'Activity_ID',
                                                    'Case_ID',
                                                    'Alias',
                                                    'Agent_distributed',
                                                    'Distributed_date',
                                                    'Distributed_hour',
                                                    'Queue',
                                                    'Completed',
                                                    'Agent_completed',
                                                    'Completed_date',
                                                    'Completed_hour',
                                                    'Type',
                                                    'Subtype',
                                                    'From_mail',
                                                    'Due_on_date',
                                                    'Due_on_hour',
                                                    'IN_OUT',
                                                    'Total_time'])
                Call_db.to_gbq(destination_table='Evolution.SLA_completed',project_id='nh-cro-forecast', if_exists='append')
            else:
                Day = str(i)
                print(f"Subiendo datos del archivo de día:{Day}")
                print('SLA_completed UPLOADING')
                with fs.open('trueit_external/SLA_completed/SLA_Completed_'+Year+Month+Day+'.csv') as f:
                    doc = pd.read_csv(f,delimiter = ';', encoding='latin-1')
                File = doc
                File.columns=['group',
                            'Service',
                            'Entry_date',
                            'Entry_hour',
                            'Activity_ID',
                            'Case_ID',
                            'Alias',
                            'Agent_distributed',
                            'Distributed_date',
                            'Distributed_hour',
                            'Queue',
                            'Completed',
                            'Agent_completed',
                            'Completed_date',
                            'Completed_hour',
                            'Type',
                            'Subtype',
                            'due_on',
                            'From_mail',
                            'IN_OUT',
                            'Total_time']
                b = []
                for i in range(len(File["due_on"])):       
                    b.append(File["due_on"][i][File["due_on"][i].rfind(" ")+1:])
                File["Due_on_hour"] = b
                c = []
                for i in range(len(File["due_on"])):
                    c.append(File["due_on"][i][:File["due_on"][i].rfind(" ")])
                File["Due_on_date"] = c
                File = File.iloc[1:len(File)]
                File.drop(['group'],axis=1,inplace=True)
                File["Entry_date"] = pd.to_datetime(File["Entry_date"], format = '%d/%m/%Y').dt.strftime('%Y-%m-%d')
                File["Completed_date"] = pd.to_datetime(File["Completed_date"], format = '%d/%m/%Y').dt.strftime('%Y-%m-%d')
                File["Distributed_date"] = pd.to_datetime(File["Distributed_date"], format = '%d/%m/%Y').dt.strftime('%Y-%m-%d')
                File.drop(columns=['due_on'],inplace = True)
                Output = File.astype(str)
                convert_dict = {'Service':object,
                                'Entry_date':object,
                                'Entry_hour':object,
                                'Activity_ID':object,
                                'Case_ID':object,
                                'Alias':object,
                                'Agent_distributed':object,
                                'Distributed_date':object,
                                'Distributed_hour':object,
                                'Queue':object,
                                'Completed':object,
                                'Agent_completed':object,
                                'Completed_date':object,
                                'Completed_hour':object,
                                'Type':object,
                                'Subtype':object,
                                'From_mail':object,
                                'IN_OUT':object,
                                'Total_time':object,
                                'Due_on_hour':object,
                                'Due_on_date':object} 
                Call_db = Output.astype(convert_dict)
                Call_db = Call_db.reindex(columns = ['Service',
                                                    'Entry_date',
                                                    'Entry_hour',
                                                    'Activity_ID',
                                                    'Case_ID',
                                                    'Alias',
                                                    'Agent_distributed',
                                                    'Distributed_date',
                                                    'Distributed_hour',
                                                    'Queue',
                                                    'Completed',
                                                    'Agent_completed',
                                                    'Completed_date',
                                                    'Completed_hour',
                                                    'Type',
                                                    'Subtype',
                                                    'From_mail',
                                                    'Due_on_date',
                                                    'Due_on_hour',
                                                    'IN_OUT',
                                                    'Total_time'])
                Call_db.to_gbq(destination_table='Evolution.SLA_completed',project_id='nh-cro-forecast', if_exists='append')
elif Query_number == 5:
        for i in range(int(inicio_dia),int(fin_dia)+1):
            if i <10: 
                Day = "0"+str(i)
                print(f"Subiendo datos del archivo de día:{Day}")
                print('Codifications_Mails UPLOADING')
                with fs.open('trueit_external/Mail_Codifications/Incoming_Category_'+Year+Month+Day+'.csv') as f:
                    doc = pd.read_csv(f,delimiter = ';', encoding='latin-1')
                doc.columns=['grouplevel',
                            'Group',
                            'Group_2',
                            'Agent',
                            'Client',
                            'Type',
                            'Subtype',
                            'Category',
                            'Subcategory',
                            'Queue',
                            'Created_on_date',
                            'Created_on_hour',
                            'Completed_on_date',
                            'Completed_on_hour',
                            'Activity_ID',
                            'Case_ID']
                doc["Created_on_date"] = pd.to_datetime(doc["Created_on_date"], format=('%d/%m/%Y')).dt.strftime('%Y-%m-%d')
                doc["Completed_on_date"] = pd.to_datetime(doc["Completed_on_date"], format=('%d/%m/%Y')).dt.strftime('%Y-%m-%d')
                Output = doc.astype(str)
                Output.drop(['grouplevel'],axis = 1, inplace = True)
                convert_dict = {'Group':object,
                                'Group_2':object,
                                'Agent':object,
                                'Client':object,
                                'Type':object,
                                'Subtype':object,
                                'Category':object,
                                'Subcategory':object,
                                'Queue':object,
                                'Created_on_date':object,
                                'Created_on_hour':object,
                                'Completed_on_date':object,
                                'Completed_on_hour':object,
                                'Activity_ID':object,
                                'Case_ID':object} 
                Call_db = Output.astype(convert_dict)
                Call_db = Call_db.astype(str)
                Call_db.replace(['NaN','None','NaT',' ','nan'],[None,None,None,None,None],inplace=True)
                Call_db.to_gbq(destination_table='Evolution.Codifications_Mails',project_id='nh-cro-forecast', if_exists='append')
            else:
                Day = str(i)
                print(f"Subiendo datos del archivo de día:{Day}")
                print('Codifications_Mails UPLOADING')
                with fs.open('trueit_external/Mail_Codifications/Incoming_Category_'+Year+Month+Day+'.csv') as f:
                    doc = pd.read_csv(f,delimiter = ';', encoding='latin-1')
                doc.columns=['grouplevel',
                            'Group',
                            'Group_2',
                            'Agent',
                            'Client',
                            'Type',
                            'Subtype',
                            'Category',
                            'Subcategory',
                            'Queue',
                            'Created_on_date',
                            'Created_on_hour',
                            'Completed_on_date',
                            'Completed_on_hour',
                            'Activity_ID',
                            'Case_ID']
                doc["Created_on_date"] = pd.to_datetime(doc["Created_on_date"], format=('%d/%m/%Y')).dt.strftime('%Y-%m-%d')
                doc["Completed_on_date"] = pd.to_datetime(doc["Completed_on_date"], format=('%d/%m/%Y')).dt.strftime('%Y-%m-%d')
                Output = doc.astype(str)
                Output.drop(['grouplevel'],axis = 1, inplace = True)
                convert_dict = {'Group':object,
                                'Group_2':object,
                                'Agent':object,
                                'Client':object,
                                'Type':object,
                                'Subtype':object,
                                'Category':object,
                                'Subcategory':object,
                                'Queue':object,
                                'Created_on_date':object,
                                'Created_on_hour':object,
                                'Completed_on_date':object,
                                'Completed_on_hour':object,
                                'Activity_ID':object,
                                'Case_ID':object} 
                Call_db = Output.astype(convert_dict)
                Call_db = Call_db.astype(str)
                Call_db.replace(['NaN','None','NaT',' ','nan'],[None,None,None,None,None],inplace=True)
                Call_db.to_gbq(destination_table='Evolution.Codifications_Mails',project_id='nh-cro-forecast', if_exists='append')
elif Query_number == 6:
        for i in range(int(inicio_dia),int(fin_dia)+1):
            if i <10: 
                Day = "0"+str(i)
                print(f"Subiendo datos del archivo de día:{Day}")
                print('a_Encuestas_Quality UPLOADING')
                with fs.open('trueit_external/Survey_FRONT/Survey_Front_'+Year+Month+Day+'.csv') as f:
                    doc = pd.read_csv(f,delimiter = ';', encoding='latin-1')
                doc = doc.iloc[1:,1:]
                column = ['Fecha',
                        'ID_transaccion',
                        'Service',
                        'Agent',
                        'ID_Agent',
                        'Campaign',
                        'ID_Campaign', 
                        'Language', 
                        'IVR_option', 
                        'Codification',
                        'Answer_survey_transfered', 
                        'Non_answer_survey_transfered', 
                        'Question_1',
                        'Mark_1', 
                        'Question_2', 
                        'Mark_2', 
                        'Question_3',
                        'Mark_3']
                doc.columns = column
                doc = doc.iloc[:,[0,1,2,3,4,5,6,7,8,10,11,12,13,14,15,16,17,9]]
                doc["Fecha"] = pd.to_datetime(doc["Fecha"], format=('%d/%m/%Y')).dt.strftime('%Y-%m-%d')
                doc = doc.astype(object)
                doc = doc.astype(str)
                doc["Question_1"] = doc["Question_1"].replace('nan', '')
                doc["Mark_1"] = doc["Mark_1"].replace('nan', '')
                doc["Question_2"] = doc["Question_2"].replace('nan', '')
                doc["Mark_2"] = doc["Mark_2"].replace('nan', '')
                doc["Question_3"] = doc["Question_3"].replace('nan', '')
                doc["Mark_3"] = doc["Mark_3"].replace('nan', '')
                doc.replace(['NaN','None','NaT',' ','nan'],[None,None,None,None,None], inplace=True)
                doc.to_gbq(destination_table='Evolution.a_Encuestas_Quality',project_id='nh-cro-forecast', if_exists='append')
            else:
                Day = str(i)
                print(f"Subiendo datos del archivo de día:{Day}")
                print('a_Encuestas_Quality UPLOADING')
                with fs.open('trueit_external/Survey_FRONT/Survey_Front_'+Year+Month+Day+'.csv') as f:
                    doc = pd.read_csv(f,delimiter = ';', encoding='latin-1')
                doc = doc.iloc[1:,1:]
                column = ['Fecha',
                        'ID_transaccion',
                        'Service',
                        'Agent',
                        'ID_Agent',
                        'Campaign',
                        'ID_Campaign', 
                        'Language', 
                        'IVR_option', 
                        'Codification',
                        'Answer_survey_transfered', 
                        'Non_answer_survey_transfered', 
                        'Question_1',
                        'Mark_1', 
                        'Question_2', 
                        'Mark_2', 
                        'Question_3',
                        'Mark_3']
                doc.columns = column
                doc = doc.iloc[:,[0,1,2,3,4,5,6,7,8,10,11,12,13,14,15,16,17,9]]
                doc["Fecha"] = pd.to_datetime(doc["Fecha"], format=('%d/%m/%Y')).dt.strftime('%Y-%m-%d')
                doc = doc.astype(object)
                doc = doc.astype(str)
                doc["Question_1"] = doc["Question_1"].replace('nan', '')
                doc["Mark_1"] = doc["Mark_1"].replace('nan', '')
                doc["Question_2"] = doc["Question_2"].replace('nan', '')
                doc["Mark_2"] = doc["Mark_2"].replace('nan', '')
                doc["Question_3"] = doc["Question_3"].replace('nan', '')
                doc["Mark_3"] = doc["Mark_3"].replace('nan', '')
                doc.replace(['NaN','None','NaT',' ','nan'],[None,None,None,None,None], inplace=True)
                doc.to_gbq(destination_table='Evolution.a_Encuestas_Quality',project_id='nh-cro-forecast', if_exists='append')
elif Query_number == 7:
        for i in range(int(inicio_dia),int(fin_dia)+1):
            if i <10: 
                Day = "0"+str(i)
                print(f"Subiendo datos del archivo de día:{Day}")
                print('NH_Encuestas UPLOADING')
                with fs.open('trueit_external/Survey_QUALITY/Survey_Quality_'+Year+Month+Day+'.csv') as f:
                    doc = pd.read_csv(f,delimiter = ';', encoding='latin-1')
                File = doc.iloc[0:,1:]
                File.columns = ['Date',
                                'Transaction_ID',
                                'Service',
                                'Agent',
                                'Agent_ID',
                                'Campaign',
                                'Campaign_ID',
                                'Language',
                                'IVR_option',
                                'Closure',
                                'N_survey_transfered',
                                'N_survey_transfered_non_answer',
                                'Question_1',
                                'Mark_question_1',
                                'Question_2',
                                'Mark_question_2']
                File['Date'] = pd.to_datetime(File["Date"],format='%d/%m/%Y').dt.strftime('%Y-%m-%d')
                Output = File.astype(str)
                Out_1 = Output[['Date', 
                                'Transaction_ID', 
                                'Service', 
                                'Agent', 
                                'Agent_ID',
                                'Campaign',
                                'Campaign_ID',
                                'Language',
                                'IVR_option',
                                'N_survey_transfered',
                                'N_survey_transfered_non_answer',
                                'Question_1',
                                'Mark_question_1',
                                'Question_2',
                                'Mark_question_2',
                                'Closure']]
                Out_2 = Out_1.replace(['NaN','None','NaT',' ','nan'],[None,None,None,None,None])
                Out_2.to_gbq(destination_table='Evolution.NH_Encuestas',project_id='nh-cro-forecast', if_exists='append')
            else:
                Day = str(i)
                print(f"Subiendo datos del archivo de día:{Day}")
                print('NH_Encuestas UPLOADING')
                with fs.open('trueit_external/Survey_QUALITY/Survey_Quality_'+Year+Month+Day+'.csv') as f:
                    doc = pd.read_csv(f,delimiter = ';', encoding='latin-1')
                File = doc.iloc[0:,1:]
                File.columns = ['Date',
                                'Transaction_ID',
                                'Service',
                                'Agent',
                                'Agent_ID',
                                'Campaign',
                                'Campaign_ID',
                                'Language',
                                'IVR_option',
                                'Closure',
                                'N_survey_transfered',
                                'N_survey_transfered_non_answer',
                                'Question_1',
                                'Mark_question_1',
                                'Question_2',
                                'Mark_question_2']
                File['Date'] = pd.to_datetime(File["Date"],format='%d/%m/%Y').dt.strftime('%Y-%m-%d')
                Output = File.astype(str)
                Out_1 = Output[['Date', 
                                'Transaction_ID', 
                                'Service', 
                                'Agent', 
                                'Agent_ID',
                                'Campaign',
                                'Campaign_ID',
                                'Language',
                                'IVR_option',
                                'N_survey_transfered',
                                'N_survey_transfered_non_answer',
                                'Question_1',
                                'Mark_question_1',
                                'Question_2',
                                'Mark_question_2',
                                'Closure']]
                Out_2 = Out_1.replace(['NaN','None','NaT',' ','nan'],[None,None,None,None,None])
                Out_2.to_gbq(destination_table='Evolution.NH_Encuestas',project_id='nh-cro-forecast', if_exists='append')
elif Query_number == 8:
        for i in range(int(inicio_dia),int(fin_dia)+1):
            if i <10: 
                Day = "0"+str(i)
                print(f"Subiendo datos del archivo de día:{Day}")
                print('e_3_NBA_Survey UPLOADING')
                with fs.open('trueit_external/Survey_NBA/Calls_Survey_NBA_service_'+Year+Month+Day+'.csv') as f:
                    doc = pd.read_csv(f,delimiter = ';', encoding='latin-1')
                doc = doc.iloc[0:,1:]
                doc = doc.astype(str)
                doc=doc.rename(columns={"Fecha":"Date",
                                        "Servicio":"Service",
                                        "Agente":"Agent",
                                        "Campaña":"Campaign",
                                        "Idioma":"Language",
                                        "IdTransaccion":"ID_Transaction",
                                        "ID Agente":"ID_Agent",
                                        "ID Campaña":"ID_Campaign",
                                        "IVR Option":"IVR_Option",
                                        "N encuestas transferidas":"Survey_transfered",
                                        "N encuestas transferidas NC":"Survey_transfered_not_answered",
                                        "Pregunta 1":"Question_1",
                                        "Nota Pregunta 1":"Question_1_mark",
                                        "Pregunta 2":"Question_2",
                                        "Nota Pregunta 2":"Question_2_mark",
                                        "Pregunta 3":"Question_3",
                                        "Nota Pregunta 3":"Question_3_mark"})
                doc["Date"] = pd.to_datetime(doc["Date"],format='%d/%m/%Y').dt.strftime('%Y-%m-%d')
                doc.replace(['NaN','None','NaT',' ','nan'],[None,None,None,None,None],inplace=True)
                output = doc.astype(str)
                output.to_gbq(destination_table='Evolution.e_3_NBA_Survey',project_id='nh-cro-forecast', if_exists='append')
            else:
                Day = str(i)
                print(f"Subiendo datos del archivo de día:{Day}")
                print('e_3_NBA_Survey UPLOADING')
                with fs.open('trueit_external/Survey_NBA/Calls_Survey_NBA_service_'+Year+Month+Day+'.csv') as f:
                    doc = pd.read_csv(f,delimiter = ';', encoding='latin-1')
                doc = doc.iloc[0:,1:]
                doc = doc.astype(str)
                doc=doc.rename(columns={"Fecha":"Date",
                                        "Servicio":"Service",
                                        "Agente":"Agent",
                                        "Campaña":"Campaign",
                                        "Idioma":"Language",
                                        "IdTransaccion":"ID_Transaction",
                                        "ID Agente":"ID_Agent",
                                        "ID Campaña":"ID_Campaign",
                                        "IVR Option":"IVR_Option",
                                        "N encuestas transferidas":"Survey_transfered",
                                        "N encuestas transferidas NC":"Survey_transfered_not_answered",
                                        "Pregunta 1":"Question_1",
                                        "Nota Pregunta 1":"Question_1_mark",
                                        "Pregunta 2":"Question_2",
                                        "Nota Pregunta 2":"Question_2_mark",
                                        "Pregunta 3":"Question_3",
                                        "Nota Pregunta 3":"Question_3_mark"})
                doc["Date"] = pd.to_datetime(doc["Date"],format='%d/%m/%Y').dt.strftime('%Y-%m-%d')
                doc.replace(['NaN','None','NaT',' ','nan'],[None,None,None,None,None],inplace=True)
                output = doc.astype(str)
                output.to_gbq(destination_table='Evolution.e_3_NBA_Survey',project_id='nh-cro-forecast', if_exists='append')
elif Query_number == 9:
        for i in range(int(inicio_dia),int(fin_dia)+1):
            if i <10: 
                Day = "0"+str(i)
                print(f"Subiendo datos del archivo de día:{Day}")
                print('e_4_CAT_Survey UPLOADING')
                with fs.open('trueit_external/Survey_NBA/Calls_Survey_CAT_service_'+Year+Month+Day+'.csv') as f:
                    doc = pd.read_csv(f,delimiter = ';', encoding='latin-1')  
                doc = doc.iloc[0:,1:]
                doc = doc.astype(str)
                doc=doc.rename(columns={"Fecha":"Date",
                                        "Servicio":"Service",
                                        "Agente":"Agent",
                                        "Campaña":"Campaign",
                                        "Idioma":"Language",
                                        "IdTransaccion":"ID_Transaction",
                                        "ID Agente":"ID_Agent",
                                        "ID Campaña":"ID_Campaign",
                                        "IVR Option":"IVR_Option",
                                        "N encuestas transferidas":"Survey_transfered",
                                        "N encuestas transferidas NC":"Survey_transfered_not_answered",
                                        "Pregunta 1":"Question_1",
                                        "Nota Pregunta 1":"Question_1_mark",
                                        "Pregunta 2":"Question_2",
                                        "Nota Pregunta 2":"Question_2_mark",
                                        "Pregunta 3":"Question_3",
                                        "Nota Pregunta 3":"Question_3_mark"})
                doc["Date"] = pd.to_datetime(doc["Date"],format='%d/%m/%Y').dt.strftime('%Y-%m-%d')
                doc.replace(['NaN','None','NaT',' ','nan'],[None,None,None,None,None],inplace=True)
                output = doc.astype(str)
                output.to_gbq(destination_table='Evolution.e_4_CAT_Survey',project_id='nh-cro-forecast', if_exists='append')
            else:
                Day = str(i)
                print(f"Subiendo datos del archivo de día:{Day}")
                print('e_4_CAT_Survey UPLOADING')
                with fs.open('trueit_external/Survey_NBA/Calls_Survey_CAT_service_'+Year+Month+Day+'.csv') as f:
                    doc = pd.read_csv(f,delimiter = ';', encoding='latin-1')  
                doc = doc.iloc[0:,1:]
                doc = doc.astype(str)
                doc=doc.rename(columns={"Fecha":"Date",
                                        "Servicio":"Service",
                                        "Agente":"Agent",
                                        "Campaña":"Campaign",
                                        "Idioma":"Language",
                                        "IdTransaccion":"ID_Transaction",
                                        "ID Agente":"ID_Agent",
                                        "ID Campaña":"ID_Campaign",
                                        "IVR Option":"IVR_Option",
                                        "N encuestas transferidas":"Survey_transfered",
                                        "N encuestas transferidas NC":"Survey_transfered_not_answered",
                                        "Pregunta 1":"Question_1",
                                        "Nota Pregunta 1":"Question_1_mark",
                                        "Pregunta 2":"Question_2",
                                        "Nota Pregunta 2":"Question_2_mark",
                                        "Pregunta 3":"Question_3",
                                        "Nota Pregunta 3":"Question_3_mark"})
                doc["Date"] = pd.to_datetime(doc["Date"],format='%d/%m/%Y').dt.strftime('%Y-%m-%d')
                doc.replace(['NaN','None','NaT',' ','nan'],[None,None,None,None,None],inplace=True)
                output = doc.astype(str)
                output.to_gbq(destination_table='Evolution.e_4_CAT_Survey',project_id='nh-cro-forecast', if_exists='append')
elif Query_number == 10:
        for i in range(int(inicio_dia),int(fin_dia)+1):
            if i <10: 
                Day = "0"+str(i)
                print(f"Subiendo datos del archivo de día:{Day}")
                print('e_6_Dispatching_oficial UPLOADING')
                with fs.open('trueit_external/Dispatching/Dispatching_'+Year+Month+Day+'.csv') as f:
                    doc = pd.read_csv(f,delimiter = ';', encoding='latin-1', dtype=str)
                doc = doc.astype(str)
                output = doc.astype(str)
                output.replace(['NaN','None','NaT',' ','nan'],[None,None,None,None,None],inplace=True)
                output.to_gbq(destination_table='Evolution.e_6_Dispatching_oficial',project_id='nh-cro-forecast', if_exists='append',location = 'EU') 
            else:
                Day = str(i)
                print(f"Subiendo datos del archivo de día:{Day}")
                print('e_6_Dispatching_oficial UPLOADING')
                with fs.open('trueit_external/Dispatching/Dispatching_'+Year+Month+Day+'.csv') as f:
                    doc = pd.read_csv(f,delimiter = ';', encoding='latin-1', dtype=str)
                doc = doc.astype(str)
                output = doc.astype(str)
                output.replace(['NaN','None','NaT',' ','nan'],[None,None,None,None,None],inplace=True)
                output.to_gbq(destination_table='Evolution.e_6_Dispatching_oficial',project_id='nh-cro-forecast', if_exists='append',location = 'EU') 
elif Query_number == 11:
        for i in range(int(inicio_dia),int(fin_dia)+1):
            if i <10: 
                Day = "0"+str(i)
                print(f"Subiendo datos del archivo de día:{Day}")
                print('e_1_NBA_Calls_Codification UPLOADING')
                with fs.open('trueit_external/Calls_Codifications_NBA/Calls_Codifications_NBA_'+Year+Month+Day+'.csv') as f:
                    doc = pd.read_csv(f,delimiter = ';', encoding='latin-1')
                doc = doc.iloc[0:,1:]
                doc = doc.astype(str)
                doc=doc.rename(columns={"ID Campaign":"ID_Campaign",
                                        "ivr_opt":"IVR_Option",
                                        "Your company has used NH/contract with us in the past":"Question1_NH_contract_company",
                                        "What cities do they usually travel to?":"Question2_Cities",
                                        "Are you considering a particular property or several? Which / Which ones?":"Question3_Properties",
                                        "Could you tell us in general how many overnight stays you think you would make at the end of the year with us?":"Question4_Overnight_stays_per_year",
                                        "Do you make your reservations directly or through an intermediary?":"Question5_Reservations_intermediary",
                                        "Could you tell us the name of the reservations intermediary?":"Question6_Reservations_intermediary_name",
                                        "Booking - intermediary Name":"Question7_Booking_intermediary_number",
                                        "Are you going to require meeting rooms?":"Question8_Meeting_rooms",
                                        "Do you book your events directly or through an intermediary?":"Question9_Events_intermediary",
                                        "Events - Intermediary Name":"Question10_Events_intermediary_number",
                                        "Could you tell us the name of the events intermediary?":"Question11_Events_intermediary_name",
                                        "Do you accept the conditions of the GPDR?":"Question12_GPDR_conditions"})

                doc["Date"] = pd.to_datetime(doc["Date"],format='%d/%m/%Y').dt.strftime('%Y-%m-%d')
                doc["Hour"] = pd.to_datetime(doc["Hour"]).dt.strftime('%H:%M:%S')
                doc["Transaction_time"] = pd.to_datetime(doc["Transaction_time"]).dt.strftime('%H:%M:%S')
                doc["Contact_time"] = pd.to_datetime(doc["Contact_time"]).dt.strftime('%H:%M:%S')
                doc.replace(['NaN','None','NaT',' ','nan'],[None,None,None,None,None],inplace=True)
                output = doc.astype(str)
                output.to_gbq(destination_table='Evolution.e_1_NBA_Calls_Codification',project_id='nh-cro-forecast', if_exists='append')
            else:
                Day = str(i)
                print(f"Subiendo datos del archivo de día:{Day}")
                with fs.open('trueit_external/Calls_Codifications_NBA/Calls_Codifications_NBA_'+Year+Month+Day+'.csv') as f:
                    doc = pd.read_csv(f,delimiter = ';', encoding='latin-1')
                doc = doc.iloc[0:,1:]
                doc = doc.astype(str)
                doc=doc.rename(columns={"ID Campaign":"ID_Campaign",
                                        "ivr_opt":"IVR_Option",
                                        "Your company has used NH/contract with us in the past":"Question1_NH_contract_company",
                                        "What cities do they usually travel to?":"Question2_Cities",
                                        "Are you considering a particular property or several? Which / Which ones?":"Question3_Properties",
                                        "Could you tell us in general how many overnight stays you think you would make at the end of the year with us?":"Question4_Overnight_stays_per_year",
                                        "Do you make your reservations directly or through an intermediary?":"Question5_Reservations_intermediary",
                                        "Could you tell us the name of the reservations intermediary?":"Question6_Reservations_intermediary_name",
                                        "Booking - intermediary Name":"Question7_Booking_intermediary_number",
                                        "Are you going to require meeting rooms?":"Question8_Meeting_rooms",
                                        "Do you book your events directly or through an intermediary?":"Question9_Events_intermediary",
                                        "Events - Intermediary Name":"Question10_Events_intermediary_number",
                                        "Could you tell us the name of the events intermediary?":"Question11_Events_intermediary_name",
                                        "Do you accept the conditions of the GPDR?":"Question12_GPDR_conditions"})

                doc["Date"] = pd.to_datetime(doc["Date"],format='%d/%m/%Y').dt.strftime('%Y-%m-%d')
                doc["Hour"] = pd.to_datetime(doc["Hour"]).dt.strftime('%H:%M:%S')
                doc["Transaction_time"] = pd.to_datetime(doc["Transaction_time"]).dt.strftime('%H:%M:%S')
                doc["Contact_time"] = pd.to_datetime(doc["Contact_time"]).dt.strftime('%H:%M:%S')
                doc.replace(['NaN','None','NaT',' ','nan'],[None,None,None,None,None],inplace=True)
                output = doc.astype(str)
                output.to_gbq(destination_table='Evolution.e_1_NBA_Calls_Codification',project_id='nh-cro-forecast', if_exists='append')
elif Query_number == 12:
        for i in range(int(inicio_dia),int(fin_dia)+1):
            if i <10: 
                Day = "0"+str(i)
                print(f"Subiendo datos del archivo de día:{Day}")
                print('e_2_NBA_Mails_Codification UPLOADING')
                with fs.open('trueit_external/Mails_Codifications_NBA/Mails_Codifications_NBA_'+Year+Month+Day+'.csv') as f:
                    doc = pd.read_csv(f,delimiter = ';', encoding='latin-1')
                doc = doc.iloc[0:,1:]
                doc = doc.astype(str)
                doc=doc.rename(columns={"ENTRY_DATE":"Entry_date",
                                        "ENTRY_HOUR":"Entry_hour",
                                        "ACTIVITY_ID":"Activity_ID",
                                        "CASE_ID":"Case_ID",
                                        "AGENT_DISTRIBUTED":"Agent_distributed",
                                        "DISTRIBUTED_DATE":"Distributed_date",
                                        "DISTRIBUTED_HOUR":"Distributed_hour",
                                        "QUEUE":"Queue",
                                        "COMPLETED":"Completed",
                                        "AGENT_COMPLETED":"Agent_completed",
                                        "COMPLETED_DATE":"Completed_date",
                                        "COMPLETED_HOUR":"Completed_hour",
                                        "DIRECTION":"Direction",
                                        "Your company has used NH/contract with us in the past":"Question1_NH_contract_company",
                                        "What cities do they usually travel to?":"Question2_Cities",
                                        "Are you considering a particular property or several? Which / Which ones?":"Question3_Properties",
                                        "Could you tell us in general how many overnight stays you think you would make at the end of the year with us?":"Question4_Overnight_stays_per_year",
                                        "Do you make your reservations directly or through an intermediary?":"Question5_Reservations_intermediary",
                                        "Could you tell us the name of the reservations intermediary?":"Question6_Reservations_intermediary_name",
                                        "Booking - intermediary Name":"Question7_Booking_intermediary_number",
                                        "Are you going to require meeting rooms?":"Question8_Meeting_rooms",
                                        "Do you book your events directly or through an intermediary?":"Question9_Events_intermediary",
                                        "Events - Intermediary Name":"Question10_Events_intermediary_number",
                                        "Could you tell us the name of the events intermediary?":"Question11_Events_intermediary_name",
                                        "Do you accept the conditions of the GPDR?":"Question12_GPDR_conditions"})
                doc["Entry_date"] = pd.to_datetime(doc["Entry_date"],format='%d/%m/%Y').dt.strftime('%Y-%m-%d')
                doc["Entry_hour"] = pd.to_datetime(doc["Entry_hour"]).dt.strftime('%H:%M:%S')
                doc["Distributed_date"] = pd.to_datetime(doc["Distributed_date"],format='%d/%m/%Y').dt.strftime('%Y-%m-%d')
                doc["Distributed_hour"] = pd.to_datetime(doc["Distributed_hour"]).dt.strftime('%H:%M:%S')
                doc["Completed_date"] = pd.to_datetime(doc["Completed_date"],format='%d/%m/%Y').dt.strftime('%Y-%m-%d')
                doc["Completed_hour"] = pd.to_datetime(doc["Completed_hour"]).dt.strftime('%H:%M:%S')
                doc.replace(['NaN','None','NaT',' ','nan'],[None,None,None,None,None],inplace=True)
                output = doc.astype(str)
                output.to_gbq(destination_table='Evolution.e_2_NBA_Mails_Codification',project_id='nh-cro-forecast', if_exists='append')
            else:
                Day = str(i)
                print(f"Subiendo datos del archivo de día:{Day}")
                print('e_2_NBA_Mails_Codification UPLOADING')
                with fs.open('trueit_external/Mails_Codifications_NBA/Mails_Codifications_NBA_'+Year+Month+Day+'.csv') as f:
                    doc = pd.read_csv(f,delimiter = ';', encoding='latin-1')
                doc = doc.iloc[0:,1:]
                doc = doc.astype(str)
                doc=doc.rename(columns={"ENTRY_DATE":"Entry_date",
                                        "ENTRY_HOUR":"Entry_hour",
                                        "ACTIVITY_ID":"Activity_ID",
                                        "CASE_ID":"Case_ID",
                                        "AGENT_DISTRIBUTED":"Agent_distributed",
                                        "DISTRIBUTED_DATE":"Distributed_date",
                                        "DISTRIBUTED_HOUR":"Distributed_hour",
                                        "QUEUE":"Queue",
                                        "COMPLETED":"Completed",
                                        "AGENT_COMPLETED":"Agent_completed",
                                        "COMPLETED_DATE":"Completed_date",
                                        "COMPLETED_HOUR":"Completed_hour",
                                        "DIRECTION":"Direction",
                                        "Your company has used NH/contract with us in the past":"Question1_NH_contract_company",
                                        "What cities do they usually travel to?":"Question2_Cities",
                                        "Are you considering a particular property or several? Which / Which ones?":"Question3_Properties",
                                        "Could you tell us in general how many overnight stays you think you would make at the end of the year with us?":"Question4_Overnight_stays_per_year",
                                        "Do you make your reservations directly or through an intermediary?":"Question5_Reservations_intermediary",
                                        "Could you tell us the name of the reservations intermediary?":"Question6_Reservations_intermediary_name",
                                        "Booking - intermediary Name":"Question7_Booking_intermediary_number",
                                        "Are you going to require meeting rooms?":"Question8_Meeting_rooms",
                                        "Do you book your events directly or through an intermediary?":"Question9_Events_intermediary",
                                        "Events - Intermediary Name":"Question10_Events_intermediary_number",
                                        "Could you tell us the name of the events intermediary?":"Question11_Events_intermediary_name",
                                        "Do you accept the conditions of the GPDR?":"Question12_GPDR_conditions"})
                doc["Entry_date"] = pd.to_datetime(doc["Entry_date"],format='%d/%m/%Y').dt.strftime('%Y-%m-%d')
                doc["Entry_hour"] = pd.to_datetime(doc["Entry_hour"]).dt.strftime('%H:%M:%S')
                doc["Distributed_date"] = pd.to_datetime(doc["Distributed_date"],format='%d/%m/%Y').dt.strftime('%Y-%m-%d')
                doc["Distributed_hour"] = pd.to_datetime(doc["Distributed_hour"]).dt.strftime('%H:%M:%S')
                doc["Completed_date"] = pd.to_datetime(doc["Completed_date"],format='%d/%m/%Y').dt.strftime('%Y-%m-%d')
                doc["Completed_hour"] = pd.to_datetime(doc["Completed_hour"]).dt.strftime('%H:%M:%S')
                doc.replace(['NaN','None','NaT',' ','nan'],[None,None,None,None,None],inplace=True)
                output = doc.astype(str)
                output.to_gbq(destination_table='Evolution.e_2_NBA_Mails_Codification',project_id='nh-cro-forecast', if_exists='append')
elif Query_number == 13:
        for i in range(int(inicio_dia),int(fin_dia)+1):
            if i <10: 
                Day = "0"+str(i)
                print(f"Subiendo datos del archivo de día:{Day}")
                print('d_1_Traduccion_mails UPLOADING')
                with fs.open('trueit_external/Mail_Traductions/Mails_Traducciones_'+Year+Month+Day+'.csv') as f:
                    doc = pd.read_csv(f,delimiter = ';', encoding='latin-1')
                doc = doc.iloc[0:,1:]
                doc = doc.astype(str)
                doc["Date"] = pd.to_datetime(doc["Date"],format='%d/%m/%Y').dt.strftime('%Y-%m-%d')
                doc["Time"] = pd.to_datetime(doc["Time"]).dt.strftime('%H:%M:%S')
                doc["Queue_time"] = pd.to_datetime(doc["Queue_time"]).dt.strftime('%H:%M:%S')
                doc["Agent_total_mail_time"] = pd.to_datetime(doc["Agent_total_mail_time"]).dt.strftime('%H:%M:%S')
                doc["Agent_real_mail_time"] = pd.to_datetime(doc["Agent_real_mail_time"]).dt.strftime('%H:%M:%S')
                doc["Transaction_time"] = pd.to_datetime(doc["Transaction_time"]).dt.strftime('%H:%M:%S')
                doc["Traduction_time"] = pd.to_datetime(doc["Traduction_time"]).dt.strftime('%H:%M:%S')
                doc["Agent_traduction_time"] = pd.to_datetime(doc["Agent_traduction_time"]).dt.strftime('%H:%M:%S')
                doc.replace(['NaN','None','NaT',' ','nan'],[None,None,None,None,None],inplace=True)
                output = doc.astype(str)
                output.to_gbq(destination_table='Evolution.d_1_Traduccion_mails',project_id='nh-cro-forecast', if_exists='append')
            else:
                Day = str(i)
                print(f"Subiendo datos del archivo de día:{Day}")
                print('d_1_Traduccion_mails UPLOADING')
                with fs.open('trueit_external/Mail_Traductions/Mails_Traducciones_'+Year+Month+Day+'.csv') as f:
                    doc = pd.read_csv(f,delimiter = ';', encoding='latin-1')
                doc = doc.iloc[0:,1:]
                doc = doc.astype(str)
                doc["Date"] = pd.to_datetime(doc["Date"],format='%d/%m/%Y').dt.strftime('%Y-%m-%d')
                doc["Time"] = pd.to_datetime(doc["Time"]).dt.strftime('%H:%M:%S')
                doc["Queue_time"] = pd.to_datetime(doc["Queue_time"]).dt.strftime('%H:%M:%S')
                doc["Agent_total_mail_time"] = pd.to_datetime(doc["Agent_total_mail_time"]).dt.strftime('%H:%M:%S')
                doc["Agent_real_mail_time"] = pd.to_datetime(doc["Agent_real_mail_time"]).dt.strftime('%H:%M:%S')
                doc["Transaction_time"] = pd.to_datetime(doc["Transaction_time"]).dt.strftime('%H:%M:%S')
                doc["Traduction_time"] = pd.to_datetime(doc["Traduction_time"]).dt.strftime('%H:%M:%S')
                doc["Agent_traduction_time"] = pd.to_datetime(doc["Agent_traduction_time"]).dt.strftime('%H:%M:%S')
                doc.replace(['NaN','None','NaT',' ','nan'],[None,None,None,None,None],inplace=True)
                output = doc.astype(str)
                output.to_gbq(destination_table='Evolution.d_1_Traduccion_mails',project_id='nh-cro-forecast', if_exists='append')
elif Query_number == 14:
        for i in range(int(inicio_dia),int(fin_dia)+1):
            if i <10: 
                Day = "0"+str(i)
                print(f"Subiendo datos del archivo de día:{Day}")
                print('e_9_Calls_Cod_Transaction UPLOADING')
                with fs.open('trueit_external/Calls_Codifications_Transaction/Calls_Cod_Transaction_'+Year+Month+Day+'.csv') as f:
                    doc = pd.read_csv(f,delimiter = ';', encoding='latin-1')
                doc = doc.iloc[0:,1:]
                doc = doc.astype(str)
                doc=doc.rename(columns={"ID Agent":"ID_Agent",
                                        "Campaña":"Campaign",
                                        "ID Campaña":"ID_Campaign",
                                        "LANGUAGE":"Language",
                                        "IVR OPTION":"IVR_Option",
                                        "LLAMADAS_IN":"Calls_IN",
                                        "LLAMADAS_OUT":"Calls_OUT",
                                        "T_DURACION_IN":"T_Duration_IN",
                                        "T_DURACION_OUT":"T_Duration_OUT",
                                        "T_DURACION":"Time_Duration"})
                doc = doc.astype(str)
                doc["Date"] = pd.to_datetime(doc["Date"],format="%d/%m/%Y").dt.strftime('%Y-%m-%d')
                doc["T_Duration_IN"] = pd.to_datetime(doc["T_Duration_IN"]).dt.strftime('%H:%M:%S')
                doc["T_Duration_OUT"] = pd.to_datetime(doc["T_Duration_OUT"]).dt.strftime('%H:%M:%S')
                doc["Time_Duration"] = pd.to_datetime(doc["Time_Duration"]).dt.strftime('%H:%M:%S')
                doc.replace(['NaN','None','NaT',' ','nan'],[None,None,None,None,None],inplace=True)
                output = doc.astype(str)
                output.to_gbq(destination_table='Evolution.e_9_Calls_Cod_Transaction',project_id='nh-cro-forecast', if_exists='append')
            else:
                Day = str(i)
                print(f"Subiendo datos del archivo de día:{Day}")
                print('e_9_Calls_Cod_Transaction UPLOADING')
                with fs.open('trueit_external/Calls_Codifications_Transaction/Calls_Cod_Transaction_'+Year+Month+Day+'.csv') as f:
                    doc = pd.read_csv(f,delimiter = ';', encoding='latin-1')
                doc = doc.iloc[0:,1:]
                doc = doc.astype(str)
                doc=doc.rename(columns={"ID Agent":"ID_Agent",
                                        "Campaña":"Campaign",
                                        "ID Campaña":"ID_Campaign",
                                        "LANGUAGE":"Language",
                                        "IVR OPTION":"IVR_Option",
                                        "LLAMADAS_IN":"Calls_IN",
                                        "LLAMADAS_OUT":"Calls_OUT",
                                        "T_DURACION_IN":"T_Duration_IN",
                                        "T_DURACION_OUT":"T_Duration_OUT",
                                        "T_DURACION":"Time_Duration"})
                doc = doc.astype(str)
                doc["Date"] = pd.to_datetime(doc["Date"],format="%d/%m/%Y").dt.strftime('%Y-%m-%d')
                doc["T_Duration_IN"] = pd.to_datetime(doc["T_Duration_IN"]).dt.strftime('%H:%M:%S')
                doc["T_Duration_OUT"] = pd.to_datetime(doc["T_Duration_OUT"]).dt.strftime('%H:%M:%S')
                doc["Time_Duration"] = pd.to_datetime(doc["Time_Duration"]).dt.strftime('%H:%M:%S')
                doc.replace(['NaN','None','NaT',' ','nan'],[None,None,None,None,None],inplace=True)
                output = doc.astype(str)
                output.to_gbq(destination_table='Evolution.e_9_Calls_Cod_Transaction',project_id='nh-cro-forecast', if_exists='append')
elif Query_number == 15:
        for i in range(int(inicio_dia),int(fin_dia)+1):
            if i <10: 
                Day = "0"+str(i)
                print(f"Subiendo datos del archivo de día:{Day}")
                print('e_10_Coordination_status UPLOADING')
                with fs.open('trueit_external/Coordination_breaks/Status_Coordination_breaks_'+Year+Month+Day+'.csv') as f:
                    doc = pd.read_csv(f,delimiter = ';', encoding='latin-1')       
                doc = doc.iloc[0:,1:]
                r = doc.astype(str)
                r["Date"] = pd.to_datetime(r["Date"],format='%d/%m/%Y').dt.strftime('%Y-%m-%d')
                r["Inicio_status_date"] = pd.to_datetime(r["Inicio_status"]).dt.strftime('%Y-%m-%d')
                r["Inicio_status_hour"] = pd.to_datetime(r["Inicio_status"]).dt.strftime('%H:%M:%S')
                r["End_status_date"] = pd.to_datetime(r["End_status"]).dt.strftime('%Y-%m-%d')
                r["End_status_hour"] = pd.to_datetime(r["End_status"]).dt.strftime('%H:%M:%S')
                output = r.astype(str)
                output.replace(['NaN','None','NaT',' ','nan'],[None,None,None,None,None],inplace=True)
                output.to_gbq(destination_table='Evolution.e_10_Coordination_status',project_id='nh-cro-forecast', if_exists='append')
            else:
                Day = str(i)
                print(f"Subiendo datos del archivo de día:{Day}")
                print('e_10_Coordination_status UPLOADING')
                with fs.open('trueit_external/Coordination_breaks/Status_Coordination_breaks_'+Year+Month+Day+'.csv') as f:
                    doc = pd.read_csv(f,delimiter = ';', encoding='latin-1')       
                doc = doc.iloc[0:,1:]
                r = doc.astype(str)
                r["Date"] = pd.to_datetime(r["Date"],format='%d/%m/%Y').dt.strftime('%Y-%m-%d')
                r["Inicio_status_date"] = pd.to_datetime(r["Inicio_status"]).dt.strftime('%Y-%m-%d')
                r["Inicio_status_hour"] = pd.to_datetime(r["Inicio_status"]).dt.strftime('%H:%M:%S')
                r["End_status_date"] = pd.to_datetime(r["End_status"]).dt.strftime('%Y-%m-%d')
                r["End_status_hour"] = pd.to_datetime(r["End_status"]).dt.strftime('%H:%M:%S')
                output = r.astype(str)
                output.replace(['NaN','None','NaT',' ','nan'],[None,None,None,None,None],inplace=True)
                output.to_gbq(destination_table='Evolution.e_10_Coordination_status',project_id='nh-cro-forecast', if_exists='append')
elif Query_number == 16:
        for i in range(int(inicio_dia),int(fin_dia)+1):
            if i <10: 
                Day = "0"+str(i)
                print(f"Subiendo datos del archivo de día:{Day}")
                print('e_11_CAT_Mails_Codification UPLOADING')
                with fs.open('trueit_external/Mails_Codifications_CAT/Mails_Codifications_CAT_'+Year+Month+Day+'.csv') as f:
                    doc = pd.read_csv(f,delimiter = ';', encoding='latin-1')
                doc = doc.iloc[0:,1:]
                doc = doc.astype(str)
                doc=doc.rename(columns={"Service":"Service",
                                        "ENTRY_DATE":"Entry_date",
                                        "ENTRY_HOUR":"Entry_hour",
                                        "ACTIVITY_ID":"Activity_ID",
                                        "CASE_ID":"Case_ID",
                                        "Alias":"Alias",
                                        "AGENT_DISTRIBUTED":"Agent_distributed",
                                        "DISTRIBUTED_DATE":"Distributed_date",
                                        "DISTRIBUTED_HOUR":"Distributed_hour",
                                        "QUEUE":"Queue",
                                        "COMPLETED":"Completed",
                                        "AGENT_COMPLETED":"Agent_completed",
                                        "COMPLETED_DATE":"Completed_date",
                                        "COMPLETED_HOUR":"Completed_hour",
                                        "Subtype":"Subtype",
                                        "DIRECTION":"Direction",
                                        "Your company has used NH/contract with us in the past":"Question1_NH_contract_company",
                                        "What cities do they usually travel to?":"Question2_Cities",
                                        "Are you considering a particular property or several? Which / Which ones?":"Question3_Properties",
                                        "Could you tell us in general how many overnight stays you think you would make at the end of the year with us?":"Question4_Overnight_stays_per_year",
                                        "Do you make your reservations directly or through an intermediary?":"Question5_Reservations_intermediary",
                                        "Could you tell us the name of the reservations intermediary?":"Question6_Reservations_intermediary_name",
                                        "Booking - intermediary Name":"Question7_Booking_intermediary_number",
                                        "Are you going to require meeting rooms?":"Question8_Meeting_rooms",
                                        "Do you book your events directly or through an intermediary?":"Question9_Events_intermediary",
                                        "Events - Intermediary Name":"Question10_Events_intermediary_number",
                                        "Could you tell us the name of the events intermediary?":"Question11_Events_intermediary_name",
                                        "Do you accept the conditions of the GPDR?":"Question12_GDPR_conditions",
                                        "Remarks":"Remarks",
                                        "PID empresa":"PID",
                                        "nombre empresa":"Enterprise_name",
                                        "categoria A/B/C":"Category",
                                        "terms & conditions aceptados yes/no":"Terms_and_condition",
                                        "credentials yes/no":"Credentials",
                                        "total revenue last year":"Revenue_last_year",
                                        "email":"Email",
                                    })
                doc["Entry_date"] = pd.to_datetime(doc["Entry_date"],format='%d/%m/%Y').dt.strftime('%Y-%m-%d')
                doc["Entry_hour"] = pd.to_datetime(doc["Entry_hour"]).dt.strftime('%H:%M:%S')
                doc["Distributed_date"] = pd.to_datetime(doc["Distributed_date"],format='%d/%m/%Y').dt.strftime('%Y-%m-%d')
                doc["Distributed_hour"] = pd.to_datetime(doc["Distributed_hour"]).dt.strftime('%H:%M:%S')
                doc["Completed_date"] = pd.to_datetime(doc["Completed_date"],format='%d/%m/%Y').dt.strftime('%Y-%m-%d')
                doc["Completed_hour"] = pd.to_datetime(doc["Completed_hour"]).dt.strftime('%H:%M:%S')
                doc.replace(['NaN','None','NaT',' ','nan'],[None,None,None,None,None],inplace=True)
                output = doc.astype(str)
                output.replace(['NaN','None','NaT',' ','nan'],[None,None,None,None,None],inplace=True)
                output.to_gbq(destination_table='Evolution.e_11_CAT_Mails_Codification',project_id='nh-cro-forecast', if_exists='append')
            else:
                Day = str(i)
                print(f"Subiendo datos del archivo de día:{Day}")
                print('e_11_CAT_Mails_Codification UPLOADING')
                with fs.open('trueit_external/Mails_Codifications_CAT/Mails_Codifications_CAT_'+Year+Month+Day+'.csv') as f:
                    doc = pd.read_csv(f,delimiter = ';', encoding='latin-1')
                doc = doc.iloc[0:,1:]
                doc = doc.astype(str)
                doc=doc.rename(columns={"Service":"Service",
                                        "ENTRY_DATE":"Entry_date",
                                        "ENTRY_HOUR":"Entry_hour",
                                        "ACTIVITY_ID":"Activity_ID",
                                        "CASE_ID":"Case_ID",
                                        "Alias":"Alias",
                                        "AGENT_DISTRIBUTED":"Agent_distributed",
                                        "DISTRIBUTED_DATE":"Distributed_date",
                                        "DISTRIBUTED_HOUR":"Distributed_hour",
                                        "QUEUE":"Queue",
                                        "COMPLETED":"Completed",
                                        "AGENT_COMPLETED":"Agent_completed",
                                        "COMPLETED_DATE":"Completed_date",
                                        "COMPLETED_HOUR":"Completed_hour",
                                        "Subtype":"Subtype",
                                        "DIRECTION":"Direction",
                                        "Your company has used NH/contract with us in the past":"Question1_NH_contract_company",
                                        "What cities do they usually travel to?":"Question2_Cities",
                                        "Are you considering a particular property or several? Which / Which ones?":"Question3_Properties",
                                        "Could you tell us in general how many overnight stays you think you would make at the end of the year with us?":"Question4_Overnight_stays_per_year",
                                        "Do you make your reservations directly or through an intermediary?":"Question5_Reservations_intermediary",
                                        "Could you tell us the name of the reservations intermediary?":"Question6_Reservations_intermediary_name",
                                        "Booking - intermediary Name":"Question7_Booking_intermediary_number",
                                        "Are you going to require meeting rooms?":"Question8_Meeting_rooms",
                                        "Do you book your events directly or through an intermediary?":"Question9_Events_intermediary",
                                        "Events - Intermediary Name":"Question10_Events_intermediary_number",
                                        "Could you tell us the name of the events intermediary?":"Question11_Events_intermediary_name",
                                        "Do you accept the conditions of the GPDR?":"Question12_GDPR_conditions",
                                        "Remarks":"Remarks",
                                        "PID empresa":"PID",
                                        "nombre empresa":"Enterprise_name",
                                        "categoria A/B/C":"Category",
                                        "terms & conditions aceptados yes/no":"Terms_and_condition",
                                        "credentials yes/no":"Credentials",
                                        "total revenue last year":"Revenue_last_year",
                                        "email":"Email",
                                    })
                doc["Entry_date"] = pd.to_datetime(doc["Entry_date"],format='%d/%m/%Y').dt.strftime('%Y-%m-%d')
                doc["Entry_hour"] = pd.to_datetime(doc["Entry_hour"]).dt.strftime('%H:%M:%S')
                doc["Distributed_date"] = pd.to_datetime(doc["Distributed_date"],format='%d/%m/%Y').dt.strftime('%Y-%m-%d')
                doc["Distributed_hour"] = pd.to_datetime(doc["Distributed_hour"]).dt.strftime('%H:%M:%S')
                doc["Completed_date"] = pd.to_datetime(doc["Completed_date"],format='%d/%m/%Y').dt.strftime('%Y-%m-%d')
                doc["Completed_hour"] = pd.to_datetime(doc["Completed_hour"]).dt.strftime('%H:%M:%S')
                doc.replace(['NaN','None','NaT',' ','nan'],[None,None,None,None,None],inplace=True)
                output = doc.astype(str)
                output.replace(['NaN','None','NaT',' ','nan'],[None,None,None,None,None],inplace=True)
                output.to_gbq(destination_table='Evolution.e_11_CAT_Mails_Codification',project_id='nh-cro-forecast', if_exists='append')
elif Query_number == 17:
        for i in range(int(inicio_dia),int(fin_dia)+1):
            if i <10: 
                Day = "0"+str(i)
                print(f"Subiendo datos del archivo de día:{Day}")
                print('e_12_CAT_Calls_Codification UPLOADING')
                with fs.open('trueit_external/Calls_Codifications_CAT/Calls_Codifications_CAT_'+Year+Month+Day+'.csv') as f:
                    doc = pd.read_csv(f,delimiter = ';', encoding='latin-1')
                doc = doc.iloc[0:,1:]
                doc = doc.astype(str)
                doc=doc.rename(columns={"Service":"Service",
                                        "Date":"Date",
                                        "Hour":"Hour",
                                        "Agent":"Agent",
                                        "ID_Agent":"Agent_ID",
                                        "Campaign":"Campaign",
                                        "ID Campaign":"ID_Campaign",
                                        "Lang":"Lang",
                                        "ivr_opt":"IVR_Option",
                                        "Type":"Type",
                                        "Transaction_time":"Transaction_time",
                                        "Contact_time":"Contact_time",
                                        "Closure":"Closure",
                                        "Your company has used NH/contract with us in the past":"Question1_NH_contract_company",
                                        "What cities do they usually travel to?":"Question2_cities",
                                        "Are you considering a particular property or several? Which / Which ones?":"Question3_Properties",
                                        "Could you tell us in general how many overnight stays you think you would make at the end of the year with us?":"Question4_Overnight_stays_per_year",
                                        "Do you make your reservations directly or through an intermediary?":"Question5_Reservations_intermediary",
                                        "Could you tell us the name of the reservations intermediary?":"Question6_Reservations_intermediary_name",
                                        "Booking - intermediary Name":"Question7_Booking_intermediary_number",
                                        "Are you going to require meeting rooms?":"Question8_Meeting_rooms",
                                        "Do you book your events directly or through an intermediary?":"Question9_Events_intermediary",
                                        "Events - Intermediary Name":"Question10_Events_intermediary_number",
                                        "Could you tell us the name of the events intermediary?":"Question11_Events_intermediary_name",
                                        "Do you accept the conditions of the GPDR?":"Question12_GDPR_conditions",
                                        "Remarks":"Remarks",
                                        "PID empresa":"PID",
                                        "nombre empresa":"Enterprise_name",
                                        "categoria A/B/C":"Category",
                                        "terms & conditions aceptados yes/no":"Terms_and_condition",
                                        "credentials yes/no":"Credentials",
                                        "total revenue last year":"Revenue_last_year",
                                        "email":"Email",
                                    })
                doc["Date"] = pd.to_datetime(doc["Date"],format='%d/%m/%Y').dt.strftime('%Y-%m-%d')
                doc["Hour"] = pd.to_datetime(doc["Hour"]).dt.strftime('%H:%M:%S')
                doc["Transaction_time"] = pd.to_datetime(doc["Transaction_time"]).dt.strftime('%H:%M:%S')
                doc["Contact_time"] = pd.to_datetime(doc["Contact_time"]).dt.strftime('%H:%M:%S')
                output = doc.astype(str)
                output.replace(['NaN','None','NaT',' ','nan'],[None,None,None,None,None],inplace=True)
                output.to_gbq(destination_table='Evolution.e_12_CAT_Calls_Codification',project_id='nh-cro-forecast', if_exists='append')
            else:
                Day = str(i)
                print(f"Subiendo datos del archivo de día:{Day}")
                print('e_12_CAT_Calls_Codification UPLOADING')
                with fs.open('trueit_external/Calls_Codifications_CAT/Calls_Codifications_CAT_'+Year+Month+Day+'.csv') as f:
                    doc = pd.read_csv(f,delimiter = ';', encoding='latin-1')
                doc = doc.iloc[0:,1:]
                doc = doc.astype(str)
                doc=doc.rename(columns={"Service":"Service",
                                        "Date":"Date",
                                        "Hour":"Hour",
                                        "Agent":"Agent",
                                        "ID_Agent":"Agent_ID",
                                        "Campaign":"Campaign",
                                        "ID Campaign":"ID_Campaign",
                                        "Lang":"Lang",
                                        "ivr_opt":"IVR_Option",
                                        "Type":"Type",
                                        "Transaction_time":"Transaction_time",
                                        "Contact_time":"Contact_time",
                                        "Closure":"Closure",
                                        "Your company has used NH/contract with us in the past":"Question1_NH_contract_company",
                                        "What cities do they usually travel to?":"Question2_cities",
                                        "Are you considering a particular property or several? Which / Which ones?":"Question3_Properties",
                                        "Could you tell us in general how many overnight stays you think you would make at the end of the year with us?":"Question4_Overnight_stays_per_year",
                                        "Do you make your reservations directly or through an intermediary?":"Question5_Reservations_intermediary",
                                        "Could you tell us the name of the reservations intermediary?":"Question6_Reservations_intermediary_name",
                                        "Booking - intermediary Name":"Question7_Booking_intermediary_number",
                                        "Are you going to require meeting rooms?":"Question8_Meeting_rooms",
                                        "Do you book your events directly or through an intermediary?":"Question9_Events_intermediary",
                                        "Events - Intermediary Name":"Question10_Events_intermediary_number",
                                        "Could you tell us the name of the events intermediary?":"Question11_Events_intermediary_name",
                                        "Do you accept the conditions of the GPDR?":"Question12_GDPR_conditions",
                                        "Remarks":"Remarks",
                                        "PID empresa":"PID",
                                        "nombre empresa":"Enterprise_name",
                                        "categoria A/B/C":"Category",
                                        "terms & conditions aceptados yes/no":"Terms_and_condition",
                                        "credentials yes/no":"Credentials",
                                        "total revenue last year":"Revenue_last_year",
                                        "email":"Email",
                                    })
                doc["Date"] = pd.to_datetime(doc["Date"],format='%d/%m/%Y').dt.strftime('%Y-%m-%d')
                doc["Hour"] = pd.to_datetime(doc["Hour"]).dt.strftime('%H:%M:%S')
                doc["Transaction_time"] = pd.to_datetime(doc["Transaction_time"]).dt.strftime('%H:%M:%S')
                doc["Contact_time"] = pd.to_datetime(doc["Contact_time"]).dt.strftime('%H:%M:%S')
                output = doc.astype(str)
                output.replace(['NaN','None','NaT',' ','nan'],[None,None,None,None,None],inplace=True)
                output.to_gbq(destination_table='Evolution.e_12_CAT_Calls_Codification',project_id='nh-cro-forecast', if_exists='append')
elif Query_number == 18:
        for i in range(int(inicio_dia),int(fin_dia)+1):
            if i <10: 
                Day = "0"+str(i)
                print(f"Subiendo datos del archivo de día:{Day}")
                print('e_13_Guest_Experience_status UPLOADING')
                with fs.open('trueit_external/Guest_experience_breaks/Status_Guest_Experience_breaks_'+Year+Month+Day+'.csv') as f:
                    doc = pd.read_csv(f,delimiter = ';', encoding='latin-1')       
                doc = doc.iloc[0:,1:]
                r = doc.astype(str)
                r["Date"] = pd.to_datetime(r["Date"],format='%d/%m/%Y').dt.strftime('%Y-%m-%d')
                r["Inicio_status_date"] = pd.to_datetime(r["Inicio_status"]).dt.strftime('%Y-%m-%d')
                r["Inicio_status_hour"] = pd.to_datetime(r["Inicio_status"]).dt.strftime('%H:%M:%S')
                r["End_status_date"] = pd.to_datetime(r["End_status"]).dt.strftime('%Y-%m-%d')
                r["End_status_hour"] = pd.to_datetime(r["End_status"]).dt.strftime('%H:%M:%S')
                output = r.astype(str)
                output.replace(['NaN','None','NaT',' ','nan'],[None,None,None,None,None],inplace=True)
                output.to_gbq(destination_table='Evolution.e_13_Guest_Experience_status',project_id='nh-cro-forecast', if_exists='append')
            else:
                Day = str(i)
                print(f"Subiendo datos del archivo de día:{Day}")
                print('e_13_Guest_Experience_status UPLOADING')
                with fs.open('trueit_external/Guest_experience_breaks/Status_Guest_Experience_breaks_'+Year+Month+Day+'.csv') as f:
                    doc = pd.read_csv(f,delimiter = ';', encoding='latin-1')       
                doc = doc.iloc[0:,1:]
                r = doc.astype(str)
                r["Date"] = pd.to_datetime(r["Date"],format='%d/%m/%Y').dt.strftime('%Y-%m-%d')
                r["Inicio_status_date"] = pd.to_datetime(r["Inicio_status"]).dt.strftime('%Y-%m-%d')
                r["Inicio_status_hour"] = pd.to_datetime(r["Inicio_status"]).dt.strftime('%H:%M:%S')
                r["End_status_date"] = pd.to_datetime(r["End_status"]).dt.strftime('%Y-%m-%d')
                r["End_status_hour"] = pd.to_datetime(r["End_status"]).dt.strftime('%H:%M:%S')
                output = r.astype(str)
                output.replace(['NaN','None','NaT',' ','nan'],[None,None,None,None,None],inplace=True)
                output.to_gbq(destination_table='Evolution.e_13_Guest_Experience_status',project_id='nh-cro-forecast', if_exists='append')
elif Query_number == 19:
        for i in range(int(inicio_dia),int(fin_dia)+1):
            if i <10: 
                Day = "0"+str(i)
                print(f"Subiendo datos del archivo de día:{Day}")
                print('e_14_waiting_time UPLOADING')
                with fs.open('trueit_external/Waiting_time/Calls_Waiting_report_'+Year+Month+Day+'.csv') as f:
                    doc = pd.read_csv(f,delimiter = ';', encoding='latin-1')
                r = doc.astype(str)
                r["Fecha"] = pd.to_datetime(r["Fecha"]).dt.strftime('%Y-%m-%d')
                r["Start_hour"] = pd.to_datetime(r["Start_hour"]).dt.strftime('%H:%M:%S')
                r["End_hour"] = pd.to_datetime(r["End_hour"]).dt.strftime('%H:%M:%S')
                r["Duration"] = pd.to_datetime(r["Duration"]).dt.strftime('%H:%M:%S')
                r["T_DBR"] = pd.to_datetime(r["T_DBR"]).dt.strftime('%H:%M:%S')
                r["T_COLA"] = pd.to_datetime(r["T_COLA"]).dt.strftime('%H:%M:%S')
                r["T_ACW"] = pd.to_datetime(r["T_ACW"]).dt.strftime('%H:%M:%S')
                output = r.astype(str)
                output.replace(['NaN','None','NaT',' ','nan'],[None,None,None,None,None],inplace=True)
                output.to_gbq(destination_table='Evolution.e_14_waiting_time',project_id='nh-cro-forecast', if_exists='append')
            else:
                Day = str(i)
                print(f"Subiendo datos del archivo de día:{Day}")
                print('e_14_waiting_time UPLOADING')
                with fs.open('trueit_external/Waiting_time/Calls_Waiting_report_'+Year+Month+Day+'.csv') as f:
                    doc = pd.read_csv(f,delimiter = ';', encoding='latin-1')
                r = doc.astype(str)
                r["Fecha"] = pd.to_datetime(r["Fecha"]).dt.strftime('%Y-%m-%d')
                r["Start_hour"] = pd.to_datetime(r["Start_hour"]).dt.strftime('%H:%M:%S')
                r["End_hour"] = pd.to_datetime(r["End_hour"]).dt.strftime('%H:%M:%S')
                r["Duration"] = pd.to_datetime(r["Duration"]).dt.strftime('%H:%M:%S')
                r["T_DBR"] = pd.to_datetime(r["T_DBR"]).dt.strftime('%H:%M:%S')
                r["T_COLA"] = pd.to_datetime(r["T_COLA"]).dt.strftime('%H:%M:%S')
                r["T_ACW"] = pd.to_datetime(r["T_ACW"]).dt.strftime('%H:%M:%S')
                output = r.astype(str)
                output.replace(['NaN','None','NaT',' ','nan'],[None,None,None,None,None],inplace=True)
                output.to_gbq(destination_table='Evolution.e_14_waiting_time',project_id='nh-cro-forecast', if_exists='append')
elif Query_number == 20:
        for i in range(int(inicio_dia),int(fin_dia)+1):
            if i <10: 
                Day = "0"+str(i)
                Day_1 = int(Day)-1
                Day_1 = "0"+str(Day_1)
                print(f"Subiendo datos del archivo de día:{Day}")
                print('e_15_agent_session_control UPLOADING')
                with fs.open('trueit_external/Agent_session_control/Status_Agent_sessions_control_'+Year+Month+Day+'.csv') as f:
                    doc = pd.read_csv(f,delimiter = ';', encoding='latin-1')      
                r = doc.astype(str)
                f = r.rename(columns={"Agent ID":"Agent_ID",
                                    "Session Start":"Session_start",
                                    "Session Available":"Session_available",
                                    "Session End":"Session_end",
                                    "Time Connected":"Time_connected",
                                    "Time Available":"Time_available",
                                    "Time Break":"Time_break",
                                    "Time Visual Rest":"Time_visual_rest"
                                    })
                f["Date"] = pd.to_datetime(f["Date"]).dt.strftime('%Y-%m-%d')
                f['Date'] = Year+"-"+Month+"-"+Day_1
                f["Session_start"] = pd.to_datetime(f["Session_start"]).dt.strftime('%Y-%m-%d %H:%M:%S')
                f["Session_available"] = pd.to_datetime(f["Session_available"]).dt.strftime('%Y-%m-%d %H:%M:%S')
                f["Session_end"] = pd.to_datetime(f["Session_end"]).dt.strftime('%Y-%m-%d %H:%M:%S')
                f["Time_connected"] = pd.to_datetime(f["Time_connected"]).dt.strftime('%H:%M:%S')
                f["Time_available"] = pd.to_datetime(f["Time_available"]).dt.strftime('%H:%M:%S')
                f["Time_break"] = pd.to_datetime(f["Time_break"]).dt.strftime('%H:%M:%S')
                output = f.astype(str)
                output.replace(['NaN','None','NaT',' ','nan'],[None,None,None,None,None],inplace=True)
                output.drop('GroupLevel', axis=1,inplace=True)
                output.to_gbq(destination_table='Evolution.e_15_agent_session_control',project_id='nh-cro-forecast', if_exists='append')
  
            else:
                Day = str(i)
                Day_1 = int(Day)-1
                Day_1 = str(Day_1)
                print(f"Subiendo datos del archivo de día:{Day}")
                print('e_15_agent_session_control UPLOADING')
                with fs.open('trueit_external/Agent_session_control/Status_Agent_sessions_control_'+Year+Month+Day+'.csv') as f:
                    doc = pd.read_csv(f,delimiter = ';', encoding='latin-1')      
                r = doc.astype(str)
                f = r.rename(columns={"Agent ID":"Agent_ID",
                                    "Session Start":"Session_start",
                                    "Session Available":"Session_available",
                                    "Session End":"Session_end",
                                    "Time Connected":"Time_connected",
                                    "Time Available":"Time_available",
                                    "Time Break":"Time_break",
                                    "Time Visual Rest":"Time_visual_rest"
                                    })
                f["Date"] = pd.to_datetime(f["Date"]).dt.strftime('%Y-%m-%d')
                f['Date'] = Year+"-"+Month+"-"+Day_1
                f["Session_start"] = pd.to_datetime(f["Session_start"]).dt.strftime('%Y-%m-%d %H:%M:%S')
                f["Session_available"] = pd.to_datetime(f["Session_available"]).dt.strftime('%Y-%m-%d %H:%M:%S')
                f["Session_end"] = pd.to_datetime(f["Session_end"]).dt.strftime('%Y-%m-%d %H:%M:%S')
                f["Time_connected"] = pd.to_datetime(f["Time_connected"]).dt.strftime('%H:%M:%S')
                f["Time_available"] = pd.to_datetime(f["Time_available"]).dt.strftime('%H:%M:%S')
                f["Time_break"] = pd.to_datetime(f["Time_break"]).dt.strftime('%H:%M:%S')
                output = f.astype(str)
                output.replace(['NaN','None','NaT',' ','nan'],[None,None,None,None,None],inplace=True)
                output.drop('GroupLevel', axis=1,inplace=True)
                output.to_gbq(destination_table='Evolution.e_15_agent_session_control',project_id='nh-cro-forecast', if_exists='append')
elif Query_number == 21:
        for i in range(int(inicio_dia),int(fin_dia)+1):
            if i <10: 
                Day = "0"+str(i)
                print(f"Subiendo datos del archivo de día:{Day}")
                print('e_16_Auditorias UPLOADING')
                with fs.open('trueit_external/Auditorias/Audit_'+Year+Month+Day+'.csv') as f:
                    doc = pd.read_csv(f,delimiter = ';', encoding='latin-1',dtype = str)      
                doc = doc.iloc[0:,1:]
                r = doc.astype(str)   
                r.replace(['NaN','None','NaT',' ','nan'],[None,None,None,None,None],inplace=True)   
                r["Event_Date"] = pd.to_datetime(r["Event_Date"],format='%d/%m/%Y').dt.strftime('%Y-%m-%d')
                r["Event_Hour"] = pd.to_datetime(r["Event_Hour"]).dt.strftime('%H:%M:%S') 
                r["Audit_Date"] = pd.to_datetime(r["Audit_Date"],format='%d/%m/%Y').dt.strftime('%Y-%m-%d')
                r["Audit_Hour"] = pd.to_datetime(r["Audit_Hour"]).dt.strftime('%H:%M:%S')   
                r["Mark"] = r["Mark"].str.replace(',','.')
                r["Mark_Factor"] = r["Mark_Factor"].str.replace(',','.')
                r["Target"] = r["Target"].str.replace(',','.')  
                r.to_gbq(destination_table='Evolution.e_16_Auditorias',project_id='nh-cro-forecast', if_exists='append')
            else:
                Day = str(i)
                print(f"Subiendo datos del archivo de día:{Day}")
                print('e_16_Auditorias UPLOADING')
                with fs.open('trueit_external/Auditorias/Audit_'+Year+Month+Day+'.csv') as f:
                    doc = pd.read_csv(f,delimiter = ';', encoding='latin-1',dtype = str)      
                doc = doc.iloc[0:,1:]
                r = doc.astype(str)   
                r.replace(['NaN','None','NaT',' ','nan'],[None,None,None,None,None],inplace=True)   
                r["Event_Date"] = pd.to_datetime(r["Event_Date"],format='%d/%m/%Y').dt.strftime('%Y-%m-%d')
                r["Event_Hour"] = pd.to_datetime(r["Event_Hour"]).dt.strftime('%H:%M:%S') 
                r["Audit_Date"] = pd.to_datetime(r["Audit_Date"],format='%d/%m/%Y').dt.strftime('%Y-%m-%d')
                r["Audit_Hour"] = pd.to_datetime(r["Audit_Hour"]).dt.strftime('%H:%M:%S')   
                r["Mark"] = r["Mark"].str.replace(',','.')
                r["Mark_Factor"] = r["Mark_Factor"].str.replace(',','.')
                r["Target"] = r["Target"].str.replace(',','.')  
                r.to_gbq(destination_table='Evolution.e_16_Auditorias',project_id='nh-cro-forecast', if_exists='append')
elif Query_number == 22:
        for i in range(int(inicio_dia),int(fin_dia)+1):
            if i <10: 
                Day = "0"+str(i)
                print(f"Subiendo datos del archivo de día:{Day}")
                print('Agents_sessions UPLOADING')
                with fs.open('trueit_external/Agent_Sessions/Agents_Sessions_'+Year+Month+Day+'.csv') as f:
                    doc = pd.read_csv(f,delimiter = ';', encoding='latin-1')      
                doc.columns = ['Date','Service','Agent','Agent_SAP','Agent_ID','Worktop','Begin_session_raw','Available_session_raw','End_session_raw','Session_duration','n_breaks','Break_reason','Break_ID','Break_duration']
                doc["Date"] = pd.to_datetime(doc["Date"], format = '%d/%m/%Y').dt.strftime('%Y-%m-%d')
                doc["Begin_session_date"] = pd.to_datetime(doc["Begin_session_raw"]).dt.strftime('%Y-%m-%d')
                doc["Begin_session_hour"] = pd.to_datetime(doc["Begin_session_raw"]).dt.strftime('%H:%M:%S')
                doc["Available_session_date"] = pd.to_datetime(doc["Available_session_raw"]).dt.strftime('%Y-%m-%d')
                doc["Available_session_hour"] = pd.to_datetime(doc["Available_session_raw"]).dt.strftime('%H:%M:%S')
                doc["End_session_date"] = pd.to_datetime(doc["End_session_raw"]).dt.strftime('%Y-%m-%d')
                doc["End_session_hour"] = pd.to_datetime(doc["End_session_raw"]).dt.strftime('%H:%M:%S')
                doc["Agents_sessions"] = pd.to_datetime(doc["Session_duration"]).dt.strftime('%H:%M:%S')
                doc["Break_duration"] = pd.to_datetime(doc["Break_duration"]).dt.strftime('%H:%M:%S')
                doc["Session_duration"] = pd.to_datetime(doc["Session_duration"]).dt.strftime('%H:%M:%S')
                Output = doc.astype(str)
                Output = Output.iloc[:,[0,1,2,3,4,5,6,14,15,7,16,17,8,18,19,9,10,11,12,13]]
                convert_dict = {'Date':object,
                                'Service':object,
                                'Agent':object,
                                'Agent_SAP':object,
                                'Agent_ID':object,
                                'Worktop':object,
                                'Begin_session_raw':object,
                                'Begin_session_date':object,
                                'Begin_session_hour':object,
                                'Available_session_raw':object,
                                'Available_session_date':object,
                                'Available_session_hour':object,
                                'End_session_raw':object,
                                'End_session_date':object,
                                'End_session_hour':object,
                                'Session_duration':object,
                                'n_breaks':object,
                                'Break_reason':object,
                                'Break_ID':object,
                                'Break_duration':object} 
                Call_db = Output.astype(convert_dict)
                Call_db = Call_db.astype(str)
                Call_db.replace(['NaN','None','NaT',' ','nan'],[None,None,None,None,None], inplace=True)
                Call_db.to_gbq(destination_table='Evolution.Agents_sessions',project_id='nh-cro-forecast', if_exists='append')
            else:
                Day = str(i)
                print(f"Subiendo datos del archivo de día:{Day}")
                print('Agents_sessions UPLOADING')
                with fs.open('trueit_external/Agent_Sessions/Agents_Sessions_'+Year+Month+Day+'.csv') as f:
                    doc = pd.read_csv(f,delimiter = ';', encoding='latin-1')      
                doc.columns = ['Date','Service','Agent','Agent_SAP','Agent_ID','Worktop','Begin_session_raw','Available_session_raw','End_session_raw','Session_duration','n_breaks','Break_reason','Break_ID','Break_duration']
                doc["Date"] = pd.to_datetime(doc["Date"], format = '%d/%m/%Y').dt.strftime('%Y-%m-%d')
                doc["Begin_session_date"] = pd.to_datetime(doc["Begin_session_raw"]).dt.strftime('%Y-%m-%d')
                doc["Begin_session_hour"] = pd.to_datetime(doc["Begin_session_raw"]).dt.strftime('%H:%M:%S')
                doc["Available_session_date"] = pd.to_datetime(doc["Available_session_raw"]).dt.strftime('%Y-%m-%d')
                doc["Available_session_hour"] = pd.to_datetime(doc["Available_session_raw"]).dt.strftime('%H:%M:%S')
                doc["End_session_date"] = pd.to_datetime(doc["End_session_raw"]).dt.strftime('%Y-%m-%d')
                doc["End_session_hour"] = pd.to_datetime(doc["End_session_raw"]).dt.strftime('%H:%M:%S')
                doc["Agents_sessions"] = pd.to_datetime(doc["Session_duration"]).dt.strftime('%H:%M:%S')
                doc["Break_duration"] = pd.to_datetime(doc["Break_duration"]).dt.strftime('%H:%M:%S')
                doc["Session_duration"] = pd.to_datetime(doc["Session_duration"]).dt.strftime('%H:%M:%S')
                Output = doc.astype(str)
                Output = Output.iloc[:,[0,1,2,3,4,5,6,14,15,7,16,17,8,18,19,9,10,11,12,13]]
                convert_dict = {'Date':object,
                                'Service':object,
                                'Agent':object,
                                'Agent_SAP':object,
                                'Agent_ID':object,
                                'Worktop':object,
                                'Begin_session_raw':object,
                                'Begin_session_date':object,
                                'Begin_session_hour':object,
                                'Available_session_raw':object,
                                'Available_session_date':object,
                                'Available_session_hour':object,
                                'End_session_raw':object,
                                'End_session_date':object,
                                'End_session_hour':object,
                                'Session_duration':object,
                                'n_breaks':object,
                                'Break_reason':object,
                                'Break_ID':object,
                                'Break_duration':object} 
                Call_db = Output.astype(convert_dict)
                Call_db = Call_db.astype(str)
                Call_db.replace(['NaN','None','NaT',' ','nan'],[None,None,None,None,None], inplace=True)
                Call_db.to_gbq(destination_table='Evolution.Agents_sessions',project_id='nh-cro-forecast', if_exists='append')
elif Query_number == 23:
        for i in range(int(inicio_dia),int(fin_dia)+1):
            if i <10: 
                Day = "0"+str(i)
                Day_1 = int(Day)-1
                Day_1 = str(Day_1)
                print(f"Subiendo datos del archivo de día:{Day}")
                print('Calls_Hotel_Distribution UPLOADING')
                with fs.open('trueit_external/Calls_Hotel_Distribution/Calls_Hotel_Distribution_'+Year+Month+Day+'.csv') as f:
                    doc = pd.read_csv(f,delimiter = ';', encoding='latin-1')      
                doc.drop('grouplevel', axis=1, inplace=True)
                column = ['Date',
                        'Service',
                        'Hotel_name',
                        'Language',
                        'IVR_option',
                        'Campaign',
                        'Campaign_ID',
                        'Slot',
                        'Incoming_calls',
                        'Answered',
                        'Non_answered',
                        'Abandoned',
                        'Abandoned_less_10s',
                        'Out_service_hours']
                doc['Date'] = Year+"-"+Month+"-"+Day_1
                doc.columns = column
                doc = doc.astype(str)
                doc.replace(['NaN','None','NaT',' ','nan'],[None,None,None,None,None],inplace=True)
                doc.to_gbq(destination_table='Evolution.Calls_Hotel_Distribution',project_id='nh-cro-forecast', if_exists='append')
            else:
                Day = str(i)
                Day_1 = int(Day)-1
                Day_1 = str(Day_1)
                print(f"Subiendo datos del archivo de día:{Day}")
                print('Calls_Hotel_Distribution UPLOADING')
                with fs.open('trueit_external/Calls_Hotel_Distribution/Calls_Hotel_Distribution_'+Year+Month+Day+'.csv') as f:
                    doc = pd.read_csv(f,delimiter = ';', encoding='latin-1')      
                doc.drop('grouplevel', axis=1, inplace=True)
                column = ['Date',
                        'Service',
                        'Hotel_name',
                        'Language',
                        'IVR_option',
                        'Campaign',
                        'Campaign_ID',
                        'Slot',
                        'Incoming_calls',
                        'Answered',
                        'Non_answered',
                        'Abandoned',
                        'Abandoned_less_10s',
                        'Out_service_hours']
                doc['Date'] = Year+"-"+Month+"-"+Day_1
                doc.columns = column
                doc = doc.astype(str)
                doc.replace(['NaN','None','NaT',' ','nan'],[None,None,None,None,None],inplace=True)
                doc.to_gbq(destination_table='Evolution.Calls_Hotel_Distribution',project_id='nh-cro-forecast', if_exists='append')
elif Query_number == 24:
        for i in range(int(inicio_dia),int(fin_dia)+1):
            if i <10: 
                Day = "0"+str(i)
                Day_1 = int(Day)-1
                Day_1 = "0"+str(Day_1)
                print(f"Subiendo datos del archivo de día:{Day}")
                print('Calls_out_of_time UPLOADING')
                with fs.open('trueit_external/Calls_out_of_Time/Call_out_of_time_'+Year+Month+Day+'.csv') as f:
                    doc = pd.read_csv(f,delimiter = ';', encoding='latin-1')
                doc = doc.iloc[1:,1:-1]
                doc['Fecha'] = Year+"-"+Month+"-"+Day_1
                column = ['Date',
                        'Service',
                        'Campaign',
                        'ID_Campaign',
                        'LANGUAGE',
                        'IVR_OPTION',
                        'Time_Slot',
                        'Total_Calls',
                        'Abandoned_Calls_less_10_secs',
                        'Abandoned_Calls_more_10_secs',
                        'Transfer_Calls',
                        'Calls_to_answering_machine']      
                doc.columns = column           
                doc = doc.astype(object)
                doc = doc.astype(str)          
                doc.replace(['NaN','None','NaT',' ','nan'],[None,None,None,None,None],inplace=True)           
                doc.to_gbq(destination_table='Evolution.Calls_out_of_time',project_id='nh-cro-forecast', if_exists='append')
            else:
                Day = str(i)
                print(f"Subiendo datos del archivo de día:{Day}")
                Day = "0"+str(i)
                Day_1 = int(Day)-1
                Day_1 = str(Day_1)
                print(f"Subiendo datos del archivo de día:{Day}")
                print('Calls_out_of_time UPLOADING')
                with fs.open('trueit_external/Calls_out_of_Time/Call_out_of_time_'+Year+Month+Day+'.csv') as f:
                    doc = pd.read_csv(f,delimiter = ';', encoding='latin-1')
                doc = doc.iloc[1:,1:-1]
                doc['Fecha'] = Year+"-"+Month+"-"+Day_1
                column = ['Date',
                        'Service',
                        'Campaign',
                        'ID_Campaign',
                        'LANGUAGE',
                        'IVR_OPTION',
                        'Time_Slot',
                        'Total_Calls',
                        'Abandoned_Calls_less_10_secs',
                        'Abandoned_Calls_more_10_secs',
                        'Transfer_Calls',
                        'Calls_to_answering_machine']      
                doc.columns = column           
                doc = doc.astype(object)
                doc = doc.astype(str)          
                doc.replace(['NaN','None','NaT',' ','nan'],[None,None,None,None,None],inplace=True)           
                doc.to_gbq(destination_table='Evolution.Calls_out_of_time',project_id='nh-cro-forecast', if_exists='append')
elif Query_number == 25:
        for i in range(int(inicio_dia),int(fin_dia)+1):
            if i <10: 
                Day = "0"+str(i)
                Day_1 = int(Day)-1
                Day_1 = "0"+str(Day_1)
                print(f"Subiendo datos del archivo de día:{Day}")
                print('IVR UPLOADING')
                with fs.open('trueit_external/IVR_Distribution/IVR_Distribution_'+Year+Month+Day+'.csv') as f:
                    doc = pd.read_csv(f,delimiter = ';', encoding='latin-1')
                doc = doc.iloc[1:,1:]
                doc['Date'] = Year+"-"+Month+"-"+Day_1
                column = ['Date', 
                        'Service',
                        'Campaign',
                        'ID',
                        'LANGUAGE',
                        'Time_Zone',
                        'IVR_OPTION',
                        'Incoming_Calls',
                        'Abandoned',
                        'Abandoned_Before_Queue',
                        'Transfered_Calls']      
                doc.columns = column         
                doc = doc.astype(object)
                doc = doc.astype(str)           
                doc.replace(['NaN','None','NaT',' ','nan'],[None,None,None,None,None],inplace=True)
                doc.to_gbq(destination_table='Evolution.IVR',project_id='nh-cro-forecast', if_exists='append')
            else:
                Day = str(i)
                print(f"Subiendo datos del archivo de día:{Day}")
                Day_1 = int(Day)-1
                Day_1 = str(Day_1)
                print(f"Subiendo datos del archivo de día:{Day}")
                print('IVR UPLOADING')
                with fs.open('trueit_external/IVR_Distribution/IVR_Distribution_'+Year+Month+Day+'.csv') as f:
                    doc = pd.read_csv(f,delimiter = ';', encoding='latin-1')
                doc = doc.iloc[1:,1:]
                doc['Date'] = Year+"-"+Month+"-"+Day_1
                column = ['Date', 
                        'Service',
                        'Campaign',
                        'ID',
                        'LANGUAGE',
                        'Time_Zone',
                        'IVR_OPTION',
                        'Incoming_Calls',
                        'Abandoned',
                        'Abandoned_Before_Queue',
                        'Transfered_Calls']      
                doc.columns = column         
                doc = doc.astype(object)
                doc = doc.astype(str)           
                doc.replace(['NaN','None','NaT',' ','nan'],[None,None,None,None,None],inplace=True)
                doc.to_gbq(destination_table='Evolution.IVR',project_id='nh-cro-forecast', if_exists='append')
elif Query_number == 26:
        for i in range(int(inicio_dia),int(fin_dia)+1):
            if i <10: 
                Day = "0"+str(i)
                print(f"Subiendo datos del archivo de día:{Day}")
                print('Rejected_calls UPLOADING')
                with fs.open('trueit_external/Calls_Rejected/Reporte_Llamadas_rechazadas_'+Year+Month+Day+'.csv') as f:
                    doc = pd.read_csv(f,delimiter = ';', encoding='latin-1')
                doc = doc.iloc[:,1:]
                doc = doc.astype(str)
                doc.columns=['Fecha',
                            'Service',
                            'Agent',
                            'Agent_ID',
                            'Campaign',
                            'Campaign_ID',
                            'Language',
                            'IVR_option',
                            'Transaction_ID',
                            'Contact_ID',
                            'Type',
                            'Time',
                            'Next_pause',
                            'Time_until_pause']
                doc["Fecha"] = pd.to_datetime(doc["Fecha"], format=('%d/%m/%Y')).dt.strftime('%Y-%m-%d')
                doc["Time"] = pd.to_datetime(doc["Time"]).dt.strftime('%H:%M:%S')
                doc["Next_pause_date"] = pd.to_datetime(doc["Next_pause"]).dt.strftime('%Y-%m-%d')
                doc["Next_pause_hour"] = pd.to_datetime(doc["Next_pause"]).dt.strftime('%H:%M:%S')
                doc["Time_until_pause"] = pd.to_datetime(doc["Time_until_pause"]).dt.strftime('%H:%M:%S')
                doc = doc[['Fecha', 
                        'Service',
                        'Agent',
                        'Agent_ID',
                        'Campaign',
                        'Campaign_ID',
                        'Language',
                        'IVR_option',
                        'Transaction_ID',
                        'Contact_ID',
                        'Type',
                        'Time',
                        'Next_pause_date',
                        'Next_pause_hour',
                        'Time_until_pause']]

                doc.replace(['NaN','None','NaT',' ','nan'],[None,None,None,None,None], inplace=True)
                doc.to_gbq(destination_table='Evolution.Rejected_calls',project_id='nh-cro-forecast', if_exists='append')
            else:
                Day = str(i)
                print(f"Subiendo datos del archivo de día:{Day}")
                print('Rejected_calls UPLOADING')
                with fs.open('trueit_external/Calls_Rejected/Reporte_Llamadas_rechazadas_'+Year+Month+Day+'.csv') as f:
                    doc = pd.read_csv(f,delimiter = ';', encoding='latin-1')
                doc = doc.iloc[:,1:]
                doc = doc.astype(str)
                doc.columns=['Fecha',
                            'Service',
                            'Agent',
                            'Agent_ID',
                            'Campaign',
                            'Campaign_ID',
                            'Language',
                            'IVR_option',
                            'Transaction_ID',
                            'Contact_ID',
                            'Type',
                            'Time',
                            'Next_pause',
                            'Time_until_pause']
                doc["Fecha"] = pd.to_datetime(doc["Fecha"], format=('%d/%m/%Y')).dt.strftime('%Y-%m-%d')
                doc["Time"] = pd.to_datetime(doc["Time"]).dt.strftime('%H:%M:%S')
                doc["Next_pause_date"] = pd.to_datetime(doc["Next_pause"]).dt.strftime('%Y-%m-%d')
                doc["Next_pause_hour"] = pd.to_datetime(doc["Next_pause"]).dt.strftime('%H:%M:%S')
                doc["Time_until_pause"] = pd.to_datetime(doc["Time_until_pause"]).dt.strftime('%H:%M:%S')
                doc = doc[['Fecha', 
                        'Service',
                        'Agent',
                        'Agent_ID',
                        'Campaign',
                        'Campaign_ID',
                        'Language',
                        'IVR_option',
                        'Transaction_ID',
                        'Contact_ID',
                        'Type',
                        'Time',
                        'Next_pause_date',
                        'Next_pause_hour',
                        'Time_until_pause']]

                doc.replace(['NaN','None','NaT',' ','nan'],[None,None,None,None,None], inplace=True)
                doc.to_gbq(destination_table='Evolution.Rejected_calls',project_id='nh-cro-forecast', if_exists='append')
elif Query_number == 27:
        for i in range(int(inicio_dia),int(fin_dia)+1):
            if i <10: 
                Day = "0"+str(i)
                print(f"Subiendo datos del archivo de día:{Day}")
                print('Calls_per_queue_time_slot UPLOADING')
                with fs.open('trueit_external/Calls_By_Agent_Campaign_Slot/CDRAgente_Campanya_Tramos_Tarde_'+Year+Month+Day+'.csv') as f:
                    doc = pd.read_csv(f,delimiter = ';', encoding='latin-1')
                doc = doc.iloc[1:,1:]
                doc = doc.astype(str)
                doc.columns=['Service', 
                            'Date',
                            'Agent',
                            'Agent_ID',
                            'Campaign',
                            'Campaign_ID',
                            'Language',
                            'IVR_option',
                            'Time_zone',
                            'Calls_in',
                            'Calls_out',
                            'Call_time_in',
                            'Call_time_out',
                            'Call_total_time']
                doc["Date"] = pd.to_datetime(doc["Date"],format='%d/%m/%Y').dt.strftime('%Y-%m-%d')
                doc["Call_time_in"] = pd.to_datetime(doc["Call_time_in"]).dt.strftime('%H:%M:%S')
                doc["Call_time_out"] = pd.to_datetime(doc["Call_time_out"]).dt.strftime('%H:%M:%S')
                doc["Call_total_time"] = pd.to_datetime(doc["Call_total_time"]).dt.strftime('%H:%M:%S')
                doc = doc[['Service', 
                        'Date',
                        'Agent',
                        'Agent_ID',
                        'Campaign',
                        'Campaign_ID',
                        'Language',
                        'IVR_option',
                        'Time_zone',
                        'Calls_in',
                        'Calls_out',
                        'Call_time_in',
                        'Call_time_out',
                        'Call_total_time']]

                doc.replace(['NaN','None','NaT',' ','nan'],[None,None,None,None,None], inplace=True)
                doc.to_gbq(destination_table='Evolution.Calls_per_queue_time_slot',project_id='nh-cro-forecast', if_exists='append')
            else:
                Day = str(i)
                print(f"Subiendo datos del archivo de día:{Day}")
                print('Calls_per_queue_time_slot UPLOADING')
                with fs.open('trueit_external/Calls_By_Agent_Campaign_Slot/CDRAgente_Campanya_Tramos_Tarde_'+Year+Month+Day+'.csv') as f:
                    doc = pd.read_csv(f,delimiter = ';', encoding='latin-1')
                doc = doc.iloc[1:,1:]
                doc = doc.astype(str)
                doc.columns=['Service', 
                            'Date',
                            'Agent',
                            'Agent_ID',
                            'Campaign',
                            'Campaign_ID',
                            'Language',
                            'IVR_option',
                            'Time_zone',
                            'Calls_in',
                            'Calls_out',
                            'Call_time_in',
                            'Call_time_out',
                            'Call_total_time']
                doc["Date"] = pd.to_datetime(doc["Date"],format='%d/%m/%Y').dt.strftime('%Y-%m-%d')
                doc["Call_time_in"] = pd.to_datetime(doc["Call_time_in"]).dt.strftime('%H:%M:%S')
                doc["Call_time_out"] = pd.to_datetime(doc["Call_time_out"]).dt.strftime('%H:%M:%S')
                doc["Call_total_time"] = pd.to_datetime(doc["Call_total_time"]).dt.strftime('%H:%M:%S')
                doc = doc[['Service', 
                        'Date',
                        'Agent',
                        'Agent_ID',
                        'Campaign',
                        'Campaign_ID',
                        'Language',
                        'IVR_option',
                        'Time_zone',
                        'Calls_in',
                        'Calls_out',
                        'Call_time_in',
                        'Call_time_out',
                        'Call_total_time']]

                doc.replace(['NaN','None','NaT',' ','nan'],[None,None,None,None,None], inplace=True)
                doc.to_gbq(destination_table='Evolution.Calls_per_queue_time_slot',project_id='nh-cro-forecast', if_exists='append')
elif Query_number == 28:
        for i in range(int(inicio_dia),int(fin_dia)+1):
            if i <10: 
                Day = "0"+str(i)
                print(f"Subiendo datos del archivo de día:{Day}")
                print('Calls_per_queue_time_slot UPLOADING')
                with fs.open('trueit_external/Calls_By_Agent_Campaign_Slot/CDRAgente_Campanya_Tramos_Maniana_'+Year+Month+Day+'.csv') as f:
                    doc = pd.read_csv(f,delimiter = ';', encoding='latin-1')
                doc = doc.iloc[1:,1:]
                doc = doc.astype(str)
                doc.columns=['Service', 
                            'Date',
                            'Agent',
                            'Agent_ID',
                            'Campaign',
                            'Campaign_ID',
                            'Language',
                            'IVR_option',
                            'Time_zone',
                            'Calls_in',
                            'Calls_out',
                            'Call_time_in',
                            'Call_time_out',
                            'Call_total_time']
                doc["Date"] = pd.to_datetime(doc["Date"],format='%d/%m/%Y').dt.strftime('%Y-%m-%d')
                doc["Call_time_in"] = pd.to_datetime(doc["Call_time_in"]).dt.strftime('%H:%M:%S')
                doc["Call_time_out"] = pd.to_datetime(doc["Call_time_out"]).dt.strftime('%H:%M:%S')
                doc["Call_total_time"] = pd.to_datetime(doc["Call_total_time"]).dt.strftime('%H:%M:%S')
                doc = doc[['Service', 
                        'Date',
                        'Agent',
                        'Agent_ID',
                        'Campaign',
                        'Campaign_ID',
                        'Language',
                        'IVR_option',
                        'Time_zone',
                        'Calls_in',
                        'Calls_out',
                        'Call_time_in',
                        'Call_time_out',
                        'Call_total_time']]

                doc.replace(['NaN','None','NaT',' ','nan'],[None,None,None,None,None], inplace=True)
                doc.to_gbq(destination_table='Evolution.Calls_per_queue_time_slot',project_id='nh-cro-forecast', if_exists='append')
            else:
                Day = str(i)
                print(f"Subiendo datos del archivo de día:{Day}")
                print('Calls_per_queue_time_slot UPLOADING')
                with fs.open('trueit_external/Calls_By_Agent_Campaign_Slot/CDRAgente_Campanya_Tramos_Maniana_'+Year+Month+Day+'.csv') as f:
                    doc = pd.read_csv(f,delimiter = ';', encoding='latin-1')
                doc = doc.iloc[1:,1:]
                doc = doc.astype(str)
                doc.columns=['Service', 
                            'Date',
                            'Agent',
                            'Agent_ID',
                            'Campaign',
                            'Campaign_ID',
                            'Language',
                            'IVR_option',
                            'Time_zone',
                            'Calls_in',
                            'Calls_out',
                            'Call_time_in',
                            'Call_time_out',
                            'Call_total_time']
                doc["Date"] = pd.to_datetime(doc["Date"],format='%d/%m/%Y').dt.strftime('%Y-%m-%d')
                doc["Call_time_in"] = pd.to_datetime(doc["Call_time_in"]).dt.strftime('%H:%M:%S')
                doc["Call_time_out"] = pd.to_datetime(doc["Call_time_out"]).dt.strftime('%H:%M:%S')
                doc["Call_total_time"] = pd.to_datetime(doc["Call_total_time"]).dt.strftime('%H:%M:%S')
                doc = doc[['Service', 
                        'Date',
                        'Agent',
                        'Agent_ID',
                        'Campaign',
                        'Campaign_ID',
                        'Language',
                        'IVR_option',
                        'Time_zone',
                        'Calls_in',
                        'Calls_out',
                        'Call_time_in',
                        'Call_time_out',
                        'Call_total_time']]

                doc.replace(['NaN','None','NaT',' ','nan'],[None,None,None,None,None], inplace=True)
                doc.to_gbq(destination_table='Evolution.Calls_per_queue_time_slot',project_id='nh-cro-forecast', if_exists='append')
elif Query_number == 29:
        for i in range(int(inicio_dia),int(fin_dia)+1):
            if i <10: 
                Day = "0"+str(i)
                print(f"Subiendo datos del archivo de día:{Day}")
                print('Codifications UPLOADING')
                with fs.open('trueit_external/Calls_Codifications/Call_Codification_'+Year+Month+Day+'.csv') as f:
                    doc = pd.read_csv(f,delimiter = ';', encoding='latin-1')           
                doc.drop('grouplevel', axis=1, inplace=True)
                doc.columns = ['Date',
                            'Service',
                            'Transaction_ID',
                            'Agent',
                            'Agent_ID',
                            'Campaign',
                            'Campaign_number',
                            'Language',
                            'IVR_option',
                            'Inbound_outbound',
                            'State',
                            'Client_type',
                            'Hotel_name',
                            'Reason_of_the_call',
                            'Reason_closure',
                            'Closure',
                            'Number_handled',
                            'Codification_time',
                            'Time_call_onhold',
                            'Time_call_active',
                            'Type_closure']  
                doc["Date"] = pd.to_datetime(doc["Date"], format='%d/%m/%Y').dt.strftime('%Y-%m-%d')
                doc = doc.astype(str)
                doc = doc[['Date',
                        'Service',
                        'Agent',
                        'Agent_ID',
                        'Transaction_ID',
                        'Campaign',
                        'Campaign_number',
                        'Language',
                        'IVR_option',
                        'Inbound_outbound',
                        'State',
                        'Client_type',
                        'Hotel_name',
                        'Reason_of_the_call',
                        'Reason_closure',
                        'Closure',
                        'Number_handled',
                        'Codification_time']]     
                Call_db = doc.astype(object)  
                Call_db['Number_handled'] = Call_db['Number_handled'].astype(int)
                Call_db.replace(['NaN','None','NaT',' ','nan'],[None,None,None,None,None], inplace=True)
                Call_db.to_gbq(destination_table='Evolution.Codifications',project_id='nh-cro-forecast', if_exists='append')
            else:
                Day = str(i)
                print(f"Subiendo datos del archivo de día:{Day}")
                print('Codifications UPLOADING')
                with fs.open('trueit_external/Calls_Codifications/Call_Codification_'+Year+Month+Day+'.csv') as f:
                    doc = pd.read_csv(f,delimiter = ';', encoding='latin-1')           
                doc.drop('grouplevel', axis=1, inplace=True)
                doc.columns = ['Date',
                            'Service',
                            'Transaction_ID',
                            'Agent',
                            'Agent_ID',
                            'Campaign',
                            'Campaign_number',
                            'Language',
                            'IVR_option',
                            'Inbound_outbound',
                            'State',
                            'Client_type',
                            'Hotel_name',
                            'Reason_of_the_call',
                            'Reason_closure',
                            'Closure',
                            'Number_handled',
                            'Codification_time',
                            'Time_call_onhold',
                            'Time_call_active',
                            'Type_closure']  
                doc["Date"] = pd.to_datetime(doc["Date"], format='%d/%m/%Y').dt.strftime('%Y-%m-%d')
                doc = doc.astype(str)
                doc = doc[['Date',
                        'Service',
                        'Agent',
                        'Agent_ID',
                        'Transaction_ID',
                        'Campaign',
                        'Campaign_number',
                        'Language',
                        'IVR_option',
                        'Inbound_outbound',
                        'State',
                        'Client_type',
                        'Hotel_name',
                        'Reason_of_the_call',
                        'Reason_closure',
                        'Closure',
                        'Number_handled',
                        'Codification_time']]     
                Call_db = doc.astype(object)  
                Call_db['Number_handled'] = Call_db['Number_handled'].astype(int)
                Call_db.replace(['NaN','None','NaT',' ','nan'],[None,None,None,None,None], inplace=True)
                Call_db.to_gbq(destination_table='Evolution.Codifications',project_id='nh-cro-forecast', if_exists='append')
elif Query_number == 30:
        for i in range(int(inicio_dia),int(fin_dia)+1):
            if i <10: 
                Day = "0"+str(i)
                print(f"Subiendo datos del archivo de día:{Day}")
                print('Agent_Nomenclature UPLOADING')
                with fs.open('trueit_external/User_Nomenclatures/Nomenclaturas_Usuarios_'+Year+Month+Day+'.csv') as f:
                    doc = pd.read_csv(f,delimiter = ';', encoding='latin-1')
                doc = doc.iloc[1:,0:]
                doc.columns = ['Date','Service', 'Agent', 'Agent_ID']
                doc["Date"] = pd.to_datetime(doc["Date"], format='%d/%m/%Y').dt.strftime('%Y-%m-%d')
                doc = doc.astype(str)
                doc.to_gbq(destination_table='Evolution.Agent_Nomenclature',project_id='nh-cro-forecast', if_exists='append')
            else:
                Day = str(i)
                print(f"Subiendo datos del archivo de día:{Day}")
                print('Agent_Nomenclature UPLOADING')
                with fs.open('trueit_external/User_Nomenclatures/Nomenclaturas_Usuarios_'+Year+Month+Day+'.csv') as f:
                    doc = pd.read_csv(f,delimiter = ';', encoding='latin-1')
                doc = doc.iloc[1:,0:]
                doc.columns = ['Date','Service', 'Agent', 'Agent_ID']
                doc["Date"] = pd.to_datetime(doc["Date"], format='%d/%m/%Y').dt.strftime('%Y-%m-%d')
                doc = doc.astype(str)
                doc.to_gbq(destination_table='Evolution.Agent_Nomenclature',project_id='nh-cro-forecast', if_exists='append')
elif Query_number == 31:
        for i in range(int(inicio_dia),int(fin_dia)+1):
            if i <10: 
                Day = "0"+str(i)
                print(f"Subiendo datos del archivo de día:{Day}")
                print('e_17_Facturacion_logins UPLOADING')
                with fs.open('trueit_external/Facturacion_current_logins/Concurrent_logins_'+Year+Month+Day+'.csv') as f:
                    doc = pd.read_csv(f,delimiter = ';', encoding='latin-1')
                doc.columns=['grouplevel',
                            'Date',
                            'Service',
                            'Total_logins',
                            'Number_concurrente',
                            'Max_concurrente']

                doc["Date"] = pd.to_datetime(doc["Date"], format = '%d/%m/%Y').dt.strftime('%Y-%m-%d')
                Output = doc.astype(str)
                Output.replace(['NaN','None','NaT',' ','nan'],[None,None,None,None,None],inplace=True)
                Output.to_gbq(destination_table='Evolution.e_17_Facturacion_logins',project_id='nh-cro-forecast', if_exists='append')
            else:
                Day = str(i)
                print(f"Subiendo datos del archivo de día:{Day}")
                print('e_17_Facturacion_logins UPLOADING')
                with fs.open('trueit_external/Facturacion_current_logins/Concurrent_logins_'+Year+Month+Day+'.csv') as f:
                    doc = pd.read_csv(f,delimiter = ';', encoding='latin-1')
                doc.columns=['grouplevel',
                            'Date',
                            'Service',
                            'Total_logins',
                            'Number_concurrente',
                            'Max_concurrente']

                doc["Date"] = pd.to_datetime(doc["Date"], format = '%d/%m/%Y').dt.strftime('%Y-%m-%d')
                Output = doc.astype(str)
                Output.replace(['NaN','None','NaT',' ','nan'],[None,None,None,None,None],inplace=True)
                Output.to_gbq(destination_table='Evolution.e_17_Facturacion_logins',project_id='nh-cro-forecast', if_exists='append')
elif Query_number == 32:
        for i in range(int(inicio_dia),int(fin_dia)+1):
            if i <10: 
                Day = "0"+str(i)
                print(f"Subiendo datos del archivo de día:{Day}")
                print('SLA_completed_Hotel_pdf UPLOADING')
                with fs.open('trueit_external/SLA_Completed_Hotel/SLA_Completed_Hotel_'+Year+Month+Day+'.csv') as f:
                    doc = pd.read_csv(f,delimiter = ';', encoding='latin-1')
                doc = doc.iloc[:,1:]
                doc.columns=['Service',
                            'Entry_date',
                            'Entry_hour',
                            'Activity_ID',
                            'Case_ID',
                            'Alias',
                            'Agent_distributed',
                            'Distributed_date',
                            'Distributed_hour',
                            'Queue',
                            'Completed',
                            'Agent_completed',
                            'Completed_date',
                            'Completed_hour',
                            'Type',
                            'Subtype',
                            'due_on',
                            'From_mail',
                            'IN_OUT',
                            'Total_time',
                            'Match',
                            'Match_level',
                            'Hotel',
                            'Country',
                            'City',
                            'Hotel_ID']
                raw = doc.astype(str)
                raw = raw.loc[:,['Service',
                        'Entry_date',
                        'Entry_hour',
                        'Activity_ID',
                        'Case_ID',
                        'Alias',
                        'Agent_distributed',
                        'Distributed_date',
                        'Distributed_hour',
                        'Queue',
                        'Completed',
                        'Agent_completed',
                        'Completed_date',
                        'Completed_hour',
                        'Type',
                        'Subtype',
                        'due_on',
                        'From_mail',
                        'IN_OUT',
                        'Total_time',
                        'Match',
                        'Match_level',
                        'Country',
                        'City',
                        'Hotel',
                        'Hotel_ID']]
                raw = doc.astype(str)
                raw.replace(['NaN','None','NaT',' ','nan'],[None,None,None,None,None],inplace=True)
                raw["Entry_date"] = pd.to_datetime(raw["Entry_date"], format = '%d/%m/%Y').dt.strftime('%Y-%m-%d')
                raw["Completed_date"] = pd.to_datetime(raw["Completed_date"], format = '%d/%m/%Y').dt.strftime('%Y-%m-%d')
                raw["Distributed_date"] = pd.to_datetime(raw["Distributed_date"], format = '%d/%m/%Y').dt.strftime('%Y-%m-%d')
                raw.to_gbq(destination_table='Evolution.SLA_completed_Hotel_pdf',project_id='nh-cro-forecast', if_exists='append')
            else:
                Day = str(i)
                print(f"Subiendo datos del archivo de día:{Day}")
                print('SLA_completed_Hotel_pdf UPLOADING')
                with fs.open('trueit_external/SLA_Completed_Hotel/SLA_Completed_Hotel_'+Year+Month+Day+'.csv') as f:
                    doc = pd.read_csv(f,delimiter = ';', encoding='latin-1')
                doc = doc.iloc[:,1:]
                doc.columns=['Service',
                            'Entry_date',
                            'Entry_hour',
                            'Activity_ID',
                            'Case_ID',
                            'Alias',
                            'Agent_distributed',
                            'Distributed_date',
                            'Distributed_hour',
                            'Queue',
                            'Completed',
                            'Agent_completed',
                            'Completed_date',
                            'Completed_hour',
                            'Type',
                            'Subtype',
                            'due_on',
                            'From_mail',
                            'IN_OUT',
                            'Total_time',
                            'Match',
                            'Match_level',
                            'Hotel',
                            'Country',
                            'City',
                            'Hotel_ID']
                raw = doc.astype(str)
                raw = raw.loc[:,['Service',
                        'Entry_date',
                        'Entry_hour',
                        'Activity_ID',
                        'Case_ID',
                        'Alias',
                        'Agent_distributed',
                        'Distributed_date',
                        'Distributed_hour',
                        'Queue',
                        'Completed',
                        'Agent_completed',
                        'Completed_date',
                        'Completed_hour',
                        'Type',
                        'Subtype',
                        'due_on',
                        'From_mail',
                        'IN_OUT',
                        'Total_time',
                        'Match',
                        'Match_level',
                        'Country',
                        'City',
                        'Hotel',
                        'Hotel_ID']]
                raw = doc.astype(str)
                raw.replace(['NaN','None','NaT',' ','nan'],[None,None,None,None,None],inplace=True)
                raw["Entry_date"] = pd.to_datetime(raw["Entry_date"], format = '%d/%m/%Y').dt.strftime('%Y-%m-%d')
                raw["Completed_date"] = pd.to_datetime(raw["Completed_date"], format = '%d/%m/%Y').dt.strftime('%Y-%m-%d')
                raw["Distributed_date"] = pd.to_datetime(raw["Distributed_date"], format = '%d/%m/%Y').dt.strftime('%Y-%m-%d')
                raw.to_gbq(destination_table='Evolution.SLA_completed_Hotel_pdf',project_id='nh-cro-forecast', if_exists='append')
elif Query_number == 33:
        for i in range(int(inicio_dia),int(fin_dia)+1):
            if i <10: 
                Day = "0"+str(i)
                print(f"Subiendo datos del archivo de día:{Day}")
                print('e_301_GEM_Audits UPLOADING')
                with fs.open('trueit_external/Auditorias/Audit_GEM_'+Year+Month+Day+'.csv') as f:
                    df_audits= pd.read_csv(f,delimiter=";", encoding='latin-1', on_bad_lines='skip', dtype='str')

                df_audits.drop('GroupLevel', inplace=True, axis=1)
                df_audits['Event_Date'] = pd.to_datetime(df_audits["Event_Date"], format = '%d/%m/%Y').dt.strftime('%Y-%m-%d')
                df_audits['Audit_Date'] = pd.to_datetime(df_audits["Audit_Date"], format = '%d/%m/%Y').dt.strftime('%Y-%m-%d')

                df_audits.replace(['NaN','None','NaT',' ','nan'],[None,None,None,None,None],inplace=True)

                df_audits.to_gbq(destination_table='Evolution.e_301_GEM_Audits',project_id='nh-cro-forecast', if_exists='append')
            else:
                Day = str(i)
                print(f"Subiendo datos del archivo de día:{Day}")
                print('e_301_GEM_Audits UPLOADING')
                with fs.open('trueit_external/Auditorias/Audit_GEM_'+Year+Month+Day+'.csv') as f:
                    df_audits= pd.read_csv(f,delimiter=";", encoding='latin-1', on_bad_lines='skip', dtype='str')

                df_audits.drop('GroupLevel', inplace=True, axis=1)
                df_audits['Event_Date'] = pd.to_datetime(df_audits["Event_Date"], format = '%d/%m/%Y').dt.strftime('%Y-%m-%d')
                df_audits['Audit_Date'] = pd.to_datetime(df_audits["Audit_Date"], format = '%d/%m/%Y').dt.strftime('%Y-%m-%d')

                df_audits.replace(['NaN','None','NaT',' ','nan'],[None,None,None,None,None],inplace=True)

                df_audits.to_gbq(destination_table='Evolution.e_301_GEM_Audits',project_id='nh-cro-forecast', if_exists='append')
else:
        for i in range(int(inicio_dia),int(fin_dia)+1):
            if i <10: 
                Day = "0"+str(i)
                print(f"Subiendo datos del archivo de día:{Day}")
                print('QA_Lista_Admin UPLOADING')
                with fs.open('trueit_external/Auditorias/Reporte_QA_Lista_Admin_'+Year+Month+Day+'.csv') as f:
                    df_audits_coord= pd.read_csv(f,delimiter=";", encoding='latin-1', on_bad_lines='skip', dtype='str')

                df_audits_coord.drop('GroupLevel',axis=1,inplace=True)
                df_audits_coord['Fecha'] = pd.to_datetime(df_audits_coord["Fecha"], format = '%d/%m/%Y').dt.strftime('%Y-%m-%d')
                df_audits_coord.to_gbq(destination_table='Evolution.QA_Lista_Admin',project_id='nh-cro-forecast', if_exists='append')
            else:
                Day = str(i)
                print(f"Subiendo datos del archivo de día:{Day}")
                print('QA_Lista_Admin UPLOADING')
                with fs.open('trueit_external/Auditorias/Reporte_QA_Lista_Admin_'+Year+Month+Day+'.csv') as f:
                    df_audits_coord= pd.read_csv(f,delimiter=";", encoding='latin-1', on_bad_lines='skip', dtype='str')

                df_audits_coord.drop('GroupLevel',axis=1,inplace=True)
                df_audits_coord['Fecha'] = pd.to_datetime(df_audits_coord["Fecha"], format = '%d/%m/%Y').dt.strftime('%Y-%m-%d')
                df_audits_coord.to_gbq(destination_table='Evolution.QA_Lista_Admin',project_id='nh-cro-forecast', if_exists='append')

Año:  2024
Mes:  2024
Día inicio: 1
Día fin: 23
Subiendo datos del archivo de día:01
Calls_per_queue UPLOADING
Subiendo datos del archivo de día:02
Calls_per_queue UPLOADING
Subiendo datos del archivo de día:03
Calls_per_queue UPLOADING
Subiendo datos del archivo de día:04
Calls_per_queue UPLOADING
Subiendo datos del archivo de día:05
Calls_per_queue UPLOADING
Subiendo datos del archivo de día:06
Calls_per_queue UPLOADING
Subiendo datos del archivo de día:07
Calls_per_queue UPLOADING
Subiendo datos del archivo de día:08
Calls_per_queue UPLOADING
Subiendo datos del archivo de día:09
Calls_per_queue UPLOADING
Subiendo datos del archivo de día:10
Calls_per_queue UPLOADING
Subiendo datos del archivo de día:11
Calls_per_queue UPLOADING
Subiendo datos del archivo de día:12
Calls_per_queue UPLOADING
Subiendo datos del archivo de día:13
Calls_per_queue UPLOADING
Subiendo datos del archivo de día:14
Calls_per_queue UPLOADING
Subiendo datos del archivo de día:15
Calls_per_queue UPLOADING
Subiend

## BORRADOR

In [1]:
import pandas as pd
import pandas_gbq
from pandas.io import gbq
from datetime import date, timedelta, datetime
import ctypes
import chardet
import gcsfs
from io import StringIO
import google.auth
from gcloud import storage
from oauth2client.service_account import ServiceAccountCredentials
from google.cloud import bigquery
from google.oauth2 import service_account
import logging
import os
import win32com.client
import warnings
warnings.filterwarnings('ignore')
import time

# Credentials to extract data from the Bucket
fs = gcsfs.GCSFileSystem(project='nh-cro-forecast', token = r'C:\Users\ej.garcia\OneDrive - Minor Hotels Europe & Americas\Escritorio\nh-cro-forecast-ec5c044f54cf.json')

#DEFINIMOS AÑO, MES E INICIO DE DIA Y FINAL DE DIA
year = "2024"
print('Año: ',year)
mes = "05"
print("Mes: ",mes)

inicio_dia = "14"
print("Día inicio:", inicio_dia)

fin_dia = "23"
print("Día fin:", fin_dia)

Año:  2024
Mes:  05
Día inicio: 14
Día fin: 23


In [10]:
with fs.open('trueit_external/SLA_Completed_Hotel/SLA_Completed_Hotel_'+year+mes+inicio_dia+'.csv') as f:
    doc = pd.read_csv(f,delimiter = ';', encoding='latin-1')

In [11]:
doc.head()

,grouplevel,Service,ENTRY_DATE,ENTRY_HOUR,ACTIVITY_ID,CASE_ID,Alias,AGENT_DISTRIBUTED,DISTRIBUTED_DATE,DISTRIBUTED_HOUR,...,Subtype,due_on,from_mail,DIRECTION,REAL_TIME_ACTIVITY,MATCH,MATCH LEVEL,PAIS,CIUDAD,HOTEL
0,0,CRO,12/05/2024,23:31:11,122175699,109038051,EDS.RESERVIERUNGEN@NH-HOTELS.COM,SYSTEM SYSTEM,12/05/2024,23:37:26,...,Finished by system,15/05/2024 01:31:11,CONNECTIVITY_SUPPORT@NH-HOTELS.COM,IN,00:00:00,at09.atter,2.0,AUSTRIA,Vienna,NH Collection Wien Zentrum
1,0,CRO,12/05/2024,23:22:45,122175656,109038230,eds.reservas@nh-hotels.com,SYSTEM SYSTEM,12/05/2024,23:38:23,...,Finish Transaction,15/05/2024 01:22:45,discovery@gha.com,IN,00:02:11,nh lyon airport,3.0,FRANCE,Lyon,NH Lyon Airport
2,0,CRO,12/05/2024,22:34:37,122175417,109038252,eds.reservas@nh-hotels.com,SYSTEM SYSTEM,12/05/2024,23:38:24,...,Finish Transaction,15/05/2024 00:34:37,discovery@gha.com,IN,00:01:33,nh lyon airport,3.0,FRANCE,Lyon,NH Lyon Airport
3,0,CRO,12/05/2024,22:39:28,122175448,109038264,eds.reservas@nh-hotels.com,SYSTEM SYSTEM,12/05/2024,23:38:24,...,Finish Transaction,15/05/2024 00:39:28,discovery@gha.com,IN,00:02:17,nh lyon airport,3.0,FRANCE,Lyon,NH Lyon Airport
4,0,CRO,12/05/2024,22:46:50,122175485,109038284,eds.reservas@nh-hotels.com,SYSTEM SYSTEM,12/05/2024,23:41:38,...,Finish Transaction,15/05/2024 00:46:50,discovery@gha.com,IN,00:00:00,nh madrid zurbano,3.0,SPAIN,Madrid,NH Zurbano


In [12]:
doc = doc.iloc[:,1:]
doc.columns=['Service',
            'Entry_date',
            'Entry_hour',
            'Activity_ID',
            'Case_ID',
            'Alias',
            'Agent_distributed',
            'Distributed_date',
            'Distributed_hour',
            'Queue',
            'Completed',
            'Agent_completed',
            'Completed_date',
            'Completed_hour',
            'Type',
            'Subtype',
            'due_on',
            'From_mail',
            'IN_OUT',
            'Total_time',
            'Match',
            'Match_level',
            'Country',
            'City',
            'Hotel'
            # 'Hotel_Id'
            ]
# doc.insert(20,'Match','KKKKK')
doc.insert(25,'Hotel_Id','')
raw = doc.astype(str)
raw = raw.loc[:,['Service',
        'Entry_date',
        'Entry_hour',
        'Activity_ID',
        'Case_ID',
        'Alias',
        'Agent_distributed',
        'Distributed_date',
        'Distributed_hour',
        'Queue',
        'Completed',
        'Agent_completed',
        'Completed_date',
        'Completed_hour',
        'Type',
        'Subtype',
        'due_on',
        'From_mail',
        'IN_OUT',
        'Total_time',
        'Match',
        'Match_level',
        'Country',
        'City',
        'Hotel',
        'Hotel_Id'
        ]]
raw.replace(['NaN','None','NaT',' ','nan'],[None,None,None,None,None],inplace=True)
raw["Entry_date"] = pd.to_datetime(raw["Entry_date"], format = '%d/%m/%Y').dt.strftime('%Y-%m-%d')
raw["Completed_date"] = pd.to_datetime(raw["Completed_date"], format = '%d/%m/%Y').dt.strftime('%Y-%m-%d')
raw["Distributed_date"] = pd.to_datetime(raw["Distributed_date"], format = '%d/%m/%Y').dt.strftime('%Y-%m-%d')

In [13]:
raw.head()

,Service,Entry_date,Entry_hour,Activity_ID,Case_ID,Alias,Agent_distributed,Distributed_date,Distributed_hour,Queue,...,due_on,From_mail,IN_OUT,Total_time,Match,Match_level,Country,City,Hotel,Hotel_Id
0,CRO,2024-05-12,23:31:11,122175699,109038051,EDS.RESERVIERUNGEN@NH-HOTELS.COM,SYSTEM SYSTEM,2024-05-12,23:37:26,EMA Failover Connectivity,...,15/05/2024 01:31:11,CONNECTIVITY_SUPPORT@NH-HOTELS.COM,IN,00:00:00,at09.atter,2.0,AUSTRIA,Vienna,NH Collection Wien Zentrum,
1,CRO,2024-05-12,23:22:45,122175656,109038230,eds.reservas@nh-hotels.com,SYSTEM SYSTEM,2024-05-12,23:38:23,EMA TPW GHA,...,15/05/2024 01:22:45,discovery@gha.com,IN,00:02:11,nh lyon airport,3.0,FRANCE,Lyon,NH Lyon Airport,
2,CRO,2024-05-12,22:34:37,122175417,109038252,eds.reservas@nh-hotels.com,SYSTEM SYSTEM,2024-05-12,23:38:24,EMA TPW GHA,...,15/05/2024 00:34:37,discovery@gha.com,IN,00:01:33,nh lyon airport,3.0,FRANCE,Lyon,NH Lyon Airport,
3,CRO,2024-05-12,22:39:28,122175448,109038264,eds.reservas@nh-hotels.com,SYSTEM SYSTEM,2024-05-12,23:38:24,EMA TPW GHA,...,15/05/2024 00:39:28,discovery@gha.com,IN,00:02:17,nh lyon airport,3.0,FRANCE,Lyon,NH Lyon Airport,
4,CRO,2024-05-12,22:46:50,122175485,109038284,eds.reservas@nh-hotels.com,SYSTEM SYSTEM,2024-05-12,23:41:38,EMA TPW GHA,...,15/05/2024 00:46:50,discovery@gha.com,IN,00:00:00,nh madrid zurbano,3.0,SPAIN,Madrid,NH Zurbano,


In [3]:
doc.head()

,grouplevel,Service,ENTRY_DATE,ENTRY_HOUR,ACTIVITY_ID,CASE_ID,Alias,AGENT_DISTRIBUTED,DISTRIBUTED_DATE,DISTRIBUTED_HOUR,...,Subtype,due_on,from_mail,DIRECTION,REAL_TIME_ACTIVITY,MATCH,MATCH LEVEL,PAIS,CIUDAD,HOTEL
0,0,CRO,12/05/2024,23:31:11,122175699,109038051,EDS.RESERVIERUNGEN@NH-HOTELS.COM,SYSTEM SYSTEM,12/05/2024,23:37:26,...,Finished by system,15/05/2024 01:31:11,CONNECTIVITY_SUPPORT@NH-HOTELS.COM,IN,00:00:00,at09.atter,2.0,AUSTRIA,Vienna,NH Collection Wien Zentrum
1,0,CRO,12/05/2024,23:22:45,122175656,109038230,eds.reservas@nh-hotels.com,SYSTEM SYSTEM,12/05/2024,23:38:23,...,Finish Transaction,15/05/2024 01:22:45,discovery@gha.com,IN,00:02:11,nh lyon airport,3.0,FRANCE,Lyon,NH Lyon Airport
2,0,CRO,12/05/2024,22:34:37,122175417,109038252,eds.reservas@nh-hotels.com,SYSTEM SYSTEM,12/05/2024,23:38:24,...,Finish Transaction,15/05/2024 00:34:37,discovery@gha.com,IN,00:01:33,nh lyon airport,3.0,FRANCE,Lyon,NH Lyon Airport
3,0,CRO,12/05/2024,22:39:28,122175448,109038264,eds.reservas@nh-hotels.com,SYSTEM SYSTEM,12/05/2024,23:38:24,...,Finish Transaction,15/05/2024 00:39:28,discovery@gha.com,IN,00:02:17,nh lyon airport,3.0,FRANCE,Lyon,NH Lyon Airport
4,0,CRO,12/05/2024,22:46:50,122175485,109038284,eds.reservas@nh-hotels.com,SYSTEM SYSTEM,12/05/2024,23:41:38,...,Finish Transaction,15/05/2024 00:46:50,discovery@gha.com,IN,00:00:00,nh madrid zurbano,3.0,SPAIN,Madrid,NH Zurbano


In [14]:
for i in range(int(inicio_dia),int(fin_dia)+1):
    if i <10: 
        Day = "0"+str(i)
        print(f"Subiendo datos del archivo de día:{Day}")
        with fs.open('trueit_external/SLA_Completed_Hotel/SLA_Completed_Hotel_'+year+mes+Day+'.csv') as f:
            doc = pd.read_csv(f,delimiter = ';', encoding='latin-1')
        doc = doc.iloc[:,1:]
        doc.columns=['Service',
                    'Entry_date',
                    'Entry_hour',
                    'Activity_ID',
                    'Case_ID',
                    'Alias',
                    'Agent_distributed',
                    'Distributed_date',
                    'Distributed_hour',
                    'Queue',
                    'Completed',
                    'Agent_completed',
                    'Completed_date',
                    'Completed_hour',
                    'Type',
                    'Subtype',
                    'due_on',
                    'From_mail',
                    'IN_OUT',
                    'Total_time',
                    'Match',
                    'Match_level',
                    'Country',
                    'City',
                    'Hotel'
                    # 'Hotel_Id'
                    ]
        # doc.insert(20,'Match','KKKKK')
        doc.insert(25,'Hotel_Id','')
        raw = doc.astype(str)
        raw = raw.loc[:,['Service',
                'Entry_date',
                'Entry_hour',
                'Activity_ID',
                'Case_ID',
                'Alias',
                'Agent_distributed',
                'Distributed_date',
                'Distributed_hour',
                'Queue',
                'Completed',
                'Agent_completed',
                'Completed_date',
                'Completed_hour',
                'Type',
                'Subtype',
                'due_on',
                'From_mail',
                'IN_OUT',
                'Total_time',
                'Match',
                'Match_level',
                'Country',
                'City',
                'Hotel',
                'Hotel_Id'
                ]]
        raw.replace(['NaN','None','NaT',' ','nan'],[None,None,None,None,None],inplace=True)
        raw["Entry_date"] = pd.to_datetime(raw["Entry_date"], format = '%d/%m/%Y').dt.strftime('%Y-%m-%d')
        raw["Completed_date"] = pd.to_datetime(raw["Completed_date"], format = '%d/%m/%Y').dt.strftime('%Y-%m-%d')
        raw["Distributed_date"] = pd.to_datetime(raw["Distributed_date"], format = '%d/%m/%Y').dt.strftime('%Y-%m-%d')
        raw.to_gbq(destination_table='Evolution.SLA_completed_Hotel_pdf',project_id='nh-cro-forecast', if_exists='append')
        time.sleep(0.5)

    else:
        Day = str(i)
        print(f"Subiendo datos del archivo de día:{Day}")
        with fs.open('trueit_external/SLA_Completed_Hotel/SLA_Completed_Hotel_'+year+mes+Day+'.csv') as f:
            doc = pd.read_csv(f,delimiter = ';', encoding='latin-1')
        doc = doc.iloc[:,1:]
        doc.columns=['Service',
                    'Entry_date',
                    'Entry_hour',
                    'Activity_ID',
                    'Case_ID',
                    'Alias',
                    'Agent_distributed',
                    'Distributed_date',
                    'Distributed_hour',
                    'Queue',
                    'Completed',
                    'Agent_completed',
                    'Completed_date',
                    'Completed_hour',
                    'Type',
                    'Subtype',
                    'due_on',
                    'From_mail',
                    'IN_OUT',
                    'Total_time',
                    'Match',
                    'Match_level',
                    'Country',
                    'City',
                    'Hotel'
                    # 'Hotel_Id'
                    ]
        # doc.insert(20,'Match','KKKKK')
        doc.insert(25,'Hotel_Id','')
        raw = doc.astype(str)
        raw = raw.loc[:,['Service',
                'Entry_date',
                'Entry_hour',
                'Activity_ID',
                'Case_ID',
                'Alias',
                'Agent_distributed',
                'Distributed_date',
                'Distributed_hour',
                'Queue',
                'Completed',
                'Agent_completed',
                'Completed_date',
                'Completed_hour',
                'Type',
                'Subtype',
                'due_on',
                'From_mail',
                'IN_OUT',
                'Total_time',
                'Match',
                'Match_level',
                'Country',
                'City',
                'Hotel',
                'Hotel_Id'
                ]]
        raw.replace(['NaN','None','NaT',' ','nan'],[None,None,None,None,None],inplace=True)
        raw["Entry_date"] = pd.to_datetime(raw["Entry_date"], format = '%d/%m/%Y').dt.strftime('%Y-%m-%d')
        raw["Completed_date"] = pd.to_datetime(raw["Completed_date"], format = '%d/%m/%Y').dt.strftime('%Y-%m-%d')
        raw["Distributed_date"] = pd.to_datetime(raw["Distributed_date"], format = '%d/%m/%Y').dt.strftime('%Y-%m-%d')
        raw.to_gbq(destination_table='Evolution.SLA_completed_Hotel_pdf',project_id='nh-cro-forecast', if_exists='append')
        time.sleep(0.5)

Subiendo datos del archivo de día:14


100%|██████████| 1/1 [00:00<?, ?it/s]


Subiendo datos del archivo de día:15


100%|██████████| 1/1 [00:00<?, ?it/s]


Subiendo datos del archivo de día:16


...

100%|██████████| 1/1 [00:00<?, ?it/s]


Subiendo datos del archivo de día:22


100%|██████████| 1/1 [00:00<?, ?it/s]


Subiendo datos del archivo de día:23


100%|██████████| 1/1 [00:00<?, ?it/s]


In [82]:
with fs.open('trueit_external/SLA_Completed_Hotel/SLA_Completed_Hotel_'+year+mes+Day+'.csv') as f:
    doc = pd.read_csv(f,delimiter = ';', encoding='latin-1')
doc = doc.iloc[:,1:]

In [83]:
doc.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11075 entries, 0 to 11074
Data columns (total 26 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   Service             11075 non-null  object 
 1   ENTRY_DATE          11075 non-null  object 
 2   ENTRY_HOUR          11075 non-null  object 
 3   ACTIVITY_ID         11075 non-null  int64  
 4   CASE_ID             11075 non-null  int64  
 5   Alias               6448 non-null   object 
 6   AGENT_DISTRIBUTED   11075 non-null  object 
 7   DISTRIBUTED_DATE    11075 non-null  object 
 8   DISTRIBUTED_HOUR    11075 non-null  object 
 9   QUEUE               11075 non-null  object 
 10  COMPLETED           11075 non-null  object 
 11  AGENT_COMPLETED     11075 non-null  object 
 12  COMPLETED_DATE      11075 non-null  object 
 13  COMPLETED_HOUR      11075 non-null  object 
 14  Type                11075 non-null  object 
 15  Subtype             11075 non-null  object 
 16  due_

In [44]:
raw.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12927 entries, 0 to 12926
Data columns (total 26 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   Service            12927 non-null  object
 1   Entry_date         12927 non-null  object
 2   Entry_hour         12927 non-null  object
 3   Activity_ID        12927 non-null  object
 4   Case_ID            12927 non-null  object
 5   Alias              7496 non-null   object
 6   Agent_distributed  12927 non-null  object
 7   Distributed_date   12927 non-null  object
 8   Distributed_hour   12927 non-null  object
 9   Queue              12927 non-null  object
 10  Completed          12927 non-null  object
 11  Agent_completed    12927 non-null  object
 12  Completed_date     12927 non-null  object
 13  Completed_hour     12927 non-null  object
 14  Type               12927 non-null  object
 15  Subtype            12927 non-null  object
 16  due_on             12927 non-null  objec

In [41]:
with fs.open('trueit_external/SLA_Completed_Hotel/SLA_Completed_Hotel_'+year+mes+Day+'.csv') as f:
    doc = pd.read_csv(f,delimiter = ';', encoding='latin-1')
doc = doc.iloc[:,1:]
doc.columns=['Service',
            'Entry_date',
            'Entry_hour',
            'Activity_ID',
            'Case_ID',
            'Alias',
            'Agent_distributed',
            'Distributed_date',
            'Distributed_hour',
            'Queue',
            'Completed',
            'Agent_completed',
            'Completed_date',
            'Completed_hour',
            'Type',
            'Subtype',
            'due_on',
            'From_mail',
            'IN_OUT',
            'Total_time',
            # 'Match',
            'Match_level',
            'Country',
            'City',
            'Hotel',
            # 'Hotel_ID'
            ]
doc.insert(20,'Match','KKKKK')
doc.insert(25,'Hotel_ID','KKKKK')


In [42]:
doc.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12927 entries, 0 to 12926
Data columns (total 26 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   Service            12927 non-null  object
 1   Entry_date         12927 non-null  object
 2   Entry_hour         12927 non-null  object
 3   Activity_ID        12927 non-null  int64 
 4   Case_ID            12927 non-null  int64 
 5   Alias              7496 non-null   object
 6   Agent_distributed  12927 non-null  object
 7   Distributed_date   12927 non-null  object
 8   Distributed_hour   12927 non-null  object
 9   Queue              12927 non-null  object
 10  Completed          12927 non-null  object
 11  Agent_completed    12927 non-null  object
 12  Completed_date     12927 non-null  object
 13  Completed_hour     12927 non-null  object
 14  Type               12927 non-null  object
 15  Subtype            12927 non-null  object
 16  due_on             12927 non-null  objec